In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
import tensorflow as tf

import random
from sklearn.model_selection import StratifiedShuffleSplit

import warnings

# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### SBERT 모델 및 벡터값 불러오기

In [ ]:
import tensorflow as tf
from sentence_transformers import SentenceTransformer
# 모델을 불러올 경로 지정
# model_path = "/content/drive/MyDrive/model_save/kobert_model1"

# 저장된 모델 불러오기
sbert_model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/model_save/SBERT_emb')
df['SBERT_emb'] = df.apply(lambda row: sbert_model.encode(row.overview), axis = 1)
# df['SBERT_emb'] = df['SBERT_emb'].apply(lambda x: np.float32(eval(x.split())))
# print(df['SBERT_emb'].shape)
df.head()

,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb
0,0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,자연,"[-0.13571063, -0.15098195, 0.95540303, 0.28600..."
1,1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,레포츠,"[-0.40310204, 0.36999023, 0.4687337, 0.0787828..."
2,2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,한식,"[-0.51864374, 0.55827594, -0.4778262, 0.194390..."
3,3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,한식,"[-0.54990906, 0.22735435, 0.010565104, 0.07347..."
4,4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,※ 영업시간 10:30 ~ 20:303대에 걸쳐 아귀만을 전문으로 취급하는 전통과 ...,음식,음식점,한식,한식,"[-0.12349912, 0.28606054, 0.24568892, -0.27898..."


In [ ]:
df.to_csv('/content/drive/MyDrive/model_save/SBERT_emb', index=False)

### doc2vec 모델 불러오기

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
# model load
doc2vec_model = Doc2Vec.load("/content/drive/MyDrive/ASAC 기업 프로젝트/개인코드/신재함/model_save/doc2vec_768_1차")

In [ ]:
#
df['doc2vec_emb'] = df['overview'].apply(lambda sentence: doc2vec_model.infer_vector(sentence.split()))
#
# doc2vec_emb = np.load('/content/drive/MyDrive/ASAC 기업 프로젝트/개인코드/신재함/model_save/doc2vec_768_1차.dv.vectors.npy')
# print(len(doc2vec_emb))

df.head()

In [ ]:
#doc2vec_emb[0]

In [ ]:
df.to_csv('/content/drive/MyDrive/ASAC 기업 프로젝트/개인코드/신재함/model_save/모델비교_df(bert1,doc2vec)', index =False)

### avg precision @k

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 벡터 전환
def bert_return_answer(question, df):
  embedding = sbert_model.encode(question)
  df['score'] = df.apply(lambda x: cos_sim(x['SBERT_emb'], embedding), axis=1)
  return df.loc[df['score'].nlargest(10).index]

def doc2vec_return_answer(question, df):
  embedding = doc2vec_model.infer_vector(question.split())
  df['doc2vec_score'] = df.apply(lambda x: cos_sim(x['embedding_doc2vec'], embedding), axis=1)
  return df.loc[df['doc2vec_score'].nlargest(5).index]

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def bert_return_answer(question, df):
  embedding = sbert_model.encode(question)
  df['score'] = df.apply(lambda x: cos_sim(x['SBERT_emb'], embedding), axis=1)
  return df.loc[df['score'].nlargest(10).index]

def weighted_avg_precision(df, question, true_label, k=10, weights=None):
    total_precision = 0

    if weights is None:
        # weights = [5/15, 4/15, 3/15, 2/15, 1/15]
        weights = [10/55, 9/55, 8/55, 7/55, 6/55, 5/55, 4/55, 3/55, 2/55, 1/55]

    total_res = bert_return_answer(question, df)
    tot_pred_labels = total_res['cat'].tolist()
    tot_correct_scores = [1 if tot_pred_labels[i] == true_label else 0 for i in range(len(tot_pred_labels))]
    tot_precision = sum([tot_correct_scores[i] * weights[i] for i in range(len(tot_pred_labels))])
    print('전체데이터 acc : ', sum(tot_correct_scores) / 10)
    print(f'전체데이터 맞춘 개수 : {sum(tot_correct_scores)}', f' / 정오표 : {tot_correct_scores}')
    print(f'전체데이터 가중치 평균 값 : {round(tot_precision, 3)}')
    print()

    for i in range(k):
        # 샘플링 0.7 비율로 k 번 진행
        sss = StratifiedShuffleSplit(n_splits=11, test_size=0.7)
        for train_index, test_index in sss.split(df, df['cat']):
          sample_df = df.iloc[test_index]

        result = bert_return_answer(question, sample_df)
        pred_labels = result['cat'].tolist()

        correct_scores = [1 if pred_labels[i] == true_label else 0 for i in range(len(pred_labels))]
        precision_at_k = sum([correct_scores[i] * weights[i] for i in range(len(pred_labels))])
        total_precision += precision_at_k

        print(f'{i+1}번째 맞춘 개수 : {sum(correct_scores)}', f' / 정오표 : {correct_scores}')
        print(f'{i+1}번째 가중치 평균 값 : {round(precision_at_k,3)}')

        res = "Weighted Average Precision@K for question: " + str(round(total_precision / k, 2))
    return res

In [ ]:
def doc2vec_weighted_avg_precision(df, question, true_label, k=10, weights=None):
    total_precision = 0

    if weights is None:
        # weights = [5/15, 4/15, 3/15, 2/15, 1/15]
        weights = [10/55, 9/55, 8/55, 7/55, 6/55, 5/55, 4/55, 3/55, 2/55, 1/55]

    total_res = doc2vec_return_answer(question, df)
    tot_pred_labels = total_res['cat'].tolist()
    tot_correct_scores = [1 if tot_pred_labels[i] == true_label else 0 for i in range(len(tot_pred_labels))]
    tot_precision = sum([tot_correct_scores[i] * weights[i] for i in range(len(tot_pred_labels))])
    print('전체데이터 acc : ', sum(tot_correct_scores) / 5)
    print(f'전체데이터 맞춘 개수 : {sum(tot_correct_scores)}', f' / 정오표 : {tot_correct_scores}')
    print(f'전체데이터 가중치 평균 값 : {round(tot_precision,3)}')
    print()

    for i in range(k):
        # 샘플링 0.7 비율로 k 번 진행
        sss = StratifiedShuffleSplit(n_splits=11, test_size=0.7)
        for train_index, test_index in sss.split(df, df['cat']):
          sample_df = df.iloc[test_index]

        result = doc2vec_return_answer(question, sample_df)
        pred_labels = result['cat'].tolist()

        correct_scores = [1 if pred_labels[i] == true_label else 0 for i in range(len(pred_labels))]
        precision_at_k = sum([correct_scores[i] * weights[i] for i in range(len(pred_labels))])
        total_precision += precision_at_k

        print(f'{i+1}번째 맞춘 개수 : {sum(correct_scores)}', f' / 정오표 : {correct_scores}')
        print(f'{i+1}번째 가중치 평균 값 : {round(precision_at_k,3)}')

        res = "Weighted Average Precision@K for question: " + str(round(total_precision / k,2))
    return res

### tests

In [ ]:
# bert ex1 : 단어1
true_label = '레포츠'
query_input = '패러글라이딩'
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

전체데이터 acc :  0.6
전체데이터 맞춘 개수 : 6  / 정오표 : [1, 0, 1, 0, 0, 1, 1, 1, 0, 1]
전체데이터 가중치 평균 값 : 0.564

1번째 맞춘 개수 : 5  / 정오표 : [1, 0, 0, 1, 0, 0, 0, 1, 1, 1]
1번째 가중치 평균 값 : 0.418
2번째 맞춘 개수 : 6  / 정오표 : [1, 0, 1, 1, 1, 1, 0, 0, 0, 1]
2번째 가중치 평균 값 : 0.673
3번째 맞춘 개수 : 5  / 정오표 : [0, 1, 0, 0, 1, 1, 1, 1, 0, 0]
3번째 가중치 평균 값 : 0.491
4번째 맞춘 개수 : 7  / 정오표 : [1, 1, 0, 1, 1, 1, 0, 1, 0, 1]
4번째 가중치 평균 값 : 0.745
5번째 맞춘 개수 : 7  / 정오표 : [1, 1, 0, 1, 1, 1, 0, 1, 1, 0]
5번째 가중치 평균 값 : 0.764
6번째 맞춘 개수 : 6  / 정오표 : [1, 0, 1, 0, 1, 1, 1, 1, 0, 0]
6번째 가중치 평균 값 : 0.655
7번째 맞춘 개수 : 5  / 정오표 : [0, 0, 1, 0, 0, 0, 1, 1, 1, 1]
7번째 가중치 평균 값 : 0.327
8번째 맞춘 개수 : 6  / 정오표 : [1, 0, 1, 1, 0, 0, 0, 1, 1, 1]
8번째 가중치 평균 값 : 0.564
9번째 맞춘 개수 : 5  / 정오표 : [1, 0, 1, 0, 0, 1, 1, 1, 0, 0]
9번째 가중치 평균 값 : 0.545
10번째 맞춘 개수 : 4  / 정오표 : [0, 0, 1, 1, 0, 1, 1, 0, 0, 0]
10번째 가중치 평균 값 : 0.436
Weighted Average Precision@K for question: 0.56


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
1933,1933,TRAIN_01933,./image/train/TRAIN_01933.jpg,"* 패러글라이더란?\n낙하산과 행글라이더의 특성을 결합한것으로 낙하산의 안정성, 분...","* 패러글라이더란?낙하산과 행글라이더의 특성을 결합한것으로 낙하산의 안정성, 분해,...",레포츠,항공 레포츠,헹글라이딩/패러글라이딩,레포츠,"[-0.56998026, 0.5367105, -0.19839369, 0.426679...",0.556451
16208,16208,TRAIN_16208,./image/train/TRAIN_16208.jpg,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.<br>,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.,음식,음식점,한식,한식,"[-0.065323845, -0.07872713, 0.74007475, -0.019...",0.545375
10614,10614,TRAIN_10614,./image/train/TRAIN_10614.jpg,드래곤레이크 CC는 하늘에서 보면 용이 입을 벌리고 있는 듯한 형상이 돋보인다. 9...,드래곤레이크 CC는 하늘에서 보면 용이 입을 벌리고 있는 듯한 형상이 돋보인다. 9...,레포츠,육상 레포츠,골프,레포츠,"[-0.41378245, 0.21954744, 0.34117657, -0.07459...",0.516429
3758,3758,TRAIN_03758,./image/train/TRAIN_03758.jpg,경남 남해군 미조면에 위치한 설리스카이워크는 원통형 구조로 360도 어디에서나 남해...,경남 남해군 미조면에 위치한 설리스카이워크는 원통형 구조로 360도 어디에서나 남해...,인문(문화/예술/역사),건축/조형물,유명건물,문화시설/건축/조형물,"[0.0035942812, 0.1639968, 0.044532776, 0.05857...",0.489604
7665,7665,TRAIN_07665,./image/train/TRAIN_07665.jpg,하늘창이 있는 무인텔이다. (1실 1주차장),하늘창이 있는 무인텔이다. (1실 1주차장),숙박,숙박시설,모텔,숙박,"[0.32552704, -0.12135628, 0.25939485, 0.076761...",0.468446
7764,7764,TRAIN_07764,./image/train/TRAIN_07764.jpg,강원도 양양 송이밸리자연휴양림 안에서 운영하는 레포츠 시설이다. ‘하늘나르기’는 짚...,강원도 양양 송이밸리자연휴양림 안에서 운영하는 레포츠 시설이다. ‘하늘나르기’는 짚...,레포츠,육상 레포츠,트래킹,레포츠,"[-0.2288569, -0.42461514, 0.42505097, 0.348536...",0.465382
3582,3582,TRAIN_03582,./image/train/TRAIN_03582.jpg,단양패러마을 패러글라이딩체험장은 항공레저(패러글라잉)의 메카로 1989년 우리나라 ...,단양패러마을 패러글라이딩체험장은 항공레저(패러글라잉)의 메카로 1989년 우리나라 ...,레포츠,항공 레포츠,헹글라이딩/패러글라이딩,레포츠,"[-0.91129047, 0.4374144, 0.3190198, 0.07108478...",0.460657
722,722,TRAIN_00722,./image/train/TRAIN_00722.jpg,강원도 평창 용평리조트 내에 자리한 마운틴코스터는 SNS에서 뜨거운 반응을 얻고 있...,강원도 평창 용평리조트 내에 자리한 마운틴코스터는 SNS에서 뜨거운 반응을 얻고 있...,레포츠,육상 레포츠,트래킹,레포츠,"[-0.6875051, -0.43424845, 0.6128622, 0.5535213...",0.458987
3030,3030,TRAIN_03030,./image/train/TRAIN_03030.jpg,홈페이지 참조,홈페이지 참조,인문(문화/예술/역사),공연/행사,기타행사,축제/공연/행사,"[0.2027652, 0.14053588, 0.6965719, -0.04203199...",0.442347
16657,16657,TRAIN_16657,./image/train/TRAIN_16657.jpg,송골매 패러글라이딩스쿨은 용인에 전용활공장을 두고 있는 패러글라이딩 스쿨이다. 서초...,송골매 패러글라이딩스쿨은 용인에 전용활공장을 두고 있는 패러글라이딩 스쿨이다. 서초...,레포츠,항공 레포츠,헹글라이딩/패러글라이딩,레포츠,"[-0.109015696, 0.3711706, 0.21279083, -0.10506...",0.429439


In [ ]:
# doc2vec ex1 : 단어1
true_label = '레포츠'
query_input = '패러글라이딩'
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.2
전체데이터 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.333

1번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.333
2번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
2번째 가중치 평균 값 : 0.333
3번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
3번째 가중치 평균 값 : 0.333
4번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
4번째 가중치 평균 값 : 0.333
5번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
5번째 가중치 평균 값 : 0.333
6번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
6번째 가중치 평균 값 : 0.333
7번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.333
8번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
8번째 가중치 평균 값 : 0.333
9번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.333
10번째 맞춘 개수 : 2  / 정오표 : [1, 0, 0, 0, 1]
10번째 가중치 평균 값 : 0.4
Weighted Average Precision@K for question: 0.34


,id,img_path,overview,cat1,cat2,cat3,score,embedding_list,cat,embedding,embedding_doc2vec,doc2vec_score
16657,TRAIN_16657,./image/train/TRAIN_16657.jpg,송골매 패러글라이딩스쿨은 용인에 전용활공장을 두고 있는 패러글라이딩 스쿨이다. 서초...,레포츠,항공 레포츠,헹글라이딩/패러글라이딩,0.106011,"[-0.10901563614606857, 0.37117114663124084, 0....",레포츠,"[-0.109015636, 0.37117115, 0.21279095, -0.1050...","[0.07870865, -0.12423689, -0.03771105, 0.00482...",0.903594
5123,TRAIN_05123,./image/train/TRAIN_05123.jpg,안동의 대표음식인 안동찜닭을 맛 볼수 있는 곳이다 대표메뉴는 찜닭이다. 경상북도 안...,음식,음식점,한식,0.119974,"[-0.5353599190711975, 0.20067697763442993, 0.2...",한식,"[-0.5353599, 0.20067698, 0.23505884, 0.1911183...","[0.15046433, -0.12288898, 0.049810253, -0.0707...",0.633344
2848,TRAIN_02848,./image/train/TRAIN_02848.jpg,50년 전통을 가지고 있는 곳이다. 경상북도 안동시에 있는 한식 전문점이다. 대표메...,음식,음식점,한식,0.191813,"[-0.18133392930030823, 0.29117968678474426, -0...",한식,"[-0.18133393, 0.2911797, -0.32618907, -0.17885...","[0.099407, -0.060256336, 0.02101707, -0.033806...",0.617686
16261,TRAIN_16261,./image/train/TRAIN_16261.jpg,봉평면에 위치하고 있는 봉평메밀닭강정은 현지인 추천 맛집이다. 메밀을 섞어 만든 닭...,음식,음식점,한식,0.315313,"[-0.003681828500702977, -0.07224772125482559, ...",한식,"[-0.0036818285, -0.07224772, -0.8921418, 0.509...","[0.15673661, -0.15800638, -0.02909074, -0.0864...",0.616337
3427,TRAIN_03427,./image/train/TRAIN_03427.jpg,큰집돌솥설렁탕의 요리는 24시간 내내 뼈를 우려내서 진한 국물 맛이 일품이다. 밥은...,음식,음식점,한식,0.260999,"[0.1126631423830986, 0.28813955187797546, -0.0...",한식,"[0.11266314, 0.28813955, -0.07313605, 0.185917...","[0.13811837, -0.10552106, -0.004729911, -0.041...",0.613394


In [ ]:
# bert ex2 : 단어 2
true_label = '한식'
query_input = '김치찌개'
res = weighted_avg_precision(df, query_input, true_label)
print(res)

display(bert_return_answer(query_input,df))

전체데이터 acc :  0.6
전체데이터 맞춘 개수 : 6  / 정오표 : [1, 1, 0, 1, 1, 0, 1, 1, 0, 0]
전체데이터 가중치 평균 값 : 0.709

1번째 맞춘 개수 : 6  / 정오표 : [1, 1, 0, 1, 1, 0, 1, 0, 0, 1]
1번째 가중치 평균 값 : 0.673
2번째 맞춘 개수 : 5  / 정오표 : [1, 1, 0, 1, 1, 0, 0, 0, 1, 0]
2번째 가중치 평균 값 : 0.618
3번째 맞춘 개수 : 7  / 정오표 : [1, 1, 0, 0, 1, 1, 0, 1, 1, 1]
3번째 가중치 평균 값 : 0.655
4번째 맞춘 개수 : 5  / 정오표 : [1, 0, 1, 1, 0, 1, 1, 0, 0, 0]
4번째 가중치 평균 값 : 0.618
5번째 맞춘 개수 : 5  / 정오표 : [1, 1, 0, 1, 1, 0, 0, 0, 0, 1]
5번째 가중치 평균 값 : 0.6
6번째 맞춘 개수 : 6  / 정오표 : [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
6번째 가중치 평균 값 : 0.6
7번째 맞춘 개수 : 5  / 정오표 : [0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
7번째 가중치 평균 값 : 0.436
8번째 맞춘 개수 : 4  / 정오표 : [1, 0, 1, 0, 1, 0, 0, 0, 0, 1]
8번째 가중치 평균 값 : 0.455
9번째 맞춘 개수 : 6  / 정오표 : [1, 1, 0, 1, 1, 0, 1, 1, 0, 0]
9번째 가중치 평균 값 : 0.709
10번째 맞춘 개수 : 6  / 정오표 : [1, 1, 0, 1, 1, 1, 0, 0, 0, 1]
10번째 가중치 평균 값 : 0.691
Weighted Average Precision@K for question: 0.61


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
16208,16208,TRAIN_16208,./image/train/TRAIN_16208.jpg,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.<br>,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.,음식,음식점,한식,한식,"[-0.065323845, -0.07872713, 0.74007475, -0.019...",0.687625
2570,2570,TRAIN_02570,./image/train/TRAIN_02570.jpg,동해시 천곡동에 위치하였으며 도루묵 전문식당으로 유명하다.,동해시 천곡동에 위치하였으며 도루묵 전문식당으로 유명하다.,음식,음식점,한식,한식,"[-0.05982764, 0.17517398, 0.53054136, -0.15591...",0.661336
3343,3343,TRAIN_03343,./image/train/TRAIN_03343.jpg,"□ 공모전명 : 전북가족영화제 공모<br>□ 주 제 : ""그런 사이""<br>□...","□ 공모전명 : 전북가족영화제 공모□ 주 제 : ""그런 사이""□ 응모자격 : ...",인문(문화/예술/역사),축제,일반축제,축제/공연/행사,"[-0.32370034, 0.34102723, 0.85591024, -0.13652...",0.616886
10361,10361,TRAIN_10361,./image/train/TRAIN_10361.jpg,"석쇠불고기, 수육, 소고기국밥, 파전 등을 전문으로 하는 한식 전문점이다.<br>","석쇠불고기, 수육, 소고기국밥, 파전 등을 전문으로 하는 한식 전문점이다.",음식,음식점,한식,한식,"[-0.18126272, 0.09312905, 0.5105129, -0.295930...",0.612120
3470,3470,TRAIN_03470,./image/train/TRAIN_03470.jpg,파주시 금촌동에 위치한 갈비전문점이다.<br>,파주시 금촌동에 위치한 갈비전문점이다.,음식,음식점,한식,한식,"[-0.22599554, -0.14217788, 0.780287, -0.379363...",0.605810
3567,3567,TRAIN_03567,./image/train/TRAIN_03567.jpg,싱싱한 기장산 해물로 요리하는 해물쟁반짜장면과 짬뽕 전문점이다.<br>,싱싱한 기장산 해물로 요리하는 해물쟁반짜장면과 짬뽕 전문점이다.,음식,음식점,중식,식사류,"[-0.28493437, -0.08427968, 0.75358915, -0.0963...",0.589156
5038,5038,TRAIN_05038,./image/train/TRAIN_05038.jpg,삼척시 정하동 새천년도로쪽에 위치하였으며 인근 연안에서 잡아올린 신선한 재료의 횟감...,삼척시 정하동 새천년도로쪽에 위치하였으며 인근 연안에서 잡아올린 신선한 재료의 횟감...,음식,음식점,한식,한식,"[-0.22012347, -0.20300132, 0.7310775, -0.23830...",0.584576
4734,4734,TRAIN_04734,./image/train/TRAIN_04734.jpg,"대형 연회석, 대형주차장 완비된 남항진동 해변가에 위치한 자연산 회전문점이다. <br>","대형 연회석, 대형주차장 완비된 남항진동 해변가에 위치한 자연산 회전문점이다.",음식,음식점,한식,한식,"[-0.25873575, -0.17379469, 0.6831365, -0.34664...",0.562918
1275,1275,TRAIN_01275,./image/train/TRAIN_01275.jpg,전통 눙경풍습(닭 둥우리 만들기/화전부치기) 재현과 민속소꿉놀이터에서 즐기는 텃밭 ...,전통 눙경풍습(닭 둥우리 만들기/화전부치기) 재현과 민속소꿉놀이터에서 즐기는 텃밭 ...,인문(문화/예술/역사),공연/행사,기타행사,축제/공연/행사,"[-0.50188535, -0.10620187, 0.70659876, -0.3598...",0.559119
3387,3387,TRAIN_03387,./image/train/TRAIN_03387.jpg,산모랭이펜션은 진부면에 위치하고 있다.,산모랭이펜션은 진부면에 위치하고 있다.,숙박,숙박시설,펜션,숙박,"[-0.068702996, -0.08789411, 0.81154406, 0.0506...",0.558199


In [ ]:
# doc2vec ex2 : 단어2
true_label = '한식'
query_input = '김치찌개'
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  1.0
전체데이터 맞춘 개수 : 5  / 정오표 : [1, 1, 1, 1, 1]
전체데이터 가중치 평균 값 : 1.0

1번째 맞춘 개수 : 3  / 정오표 : [1, 0, 1, 1, 0]
1번째 가중치 평균 값 : 0.667
2번째 맞춘 개수 : 5  / 정오표 : [1, 1, 1, 1, 1]
2번째 가중치 평균 값 : 1.0
3번째 맞춘 개수 : 5  / 정오표 : [1, 1, 1, 1, 1]
3번째 가중치 평균 값 : 1.0
4번째 맞춘 개수 : 5  / 정오표 : [1, 1, 1, 1, 1]
4번째 가중치 평균 값 : 1.0
5번째 맞춘 개수 : 4  / 정오표 : [1, 1, 1, 1, 0]
5번째 가중치 평균 값 : 0.933
6번째 맞춘 개수 : 5  / 정오표 : [1, 1, 1, 1, 1]
6번째 가중치 평균 값 : 1.0
7번째 맞춘 개수 : 5  / 정오표 : [1, 1, 1, 1, 1]
7번째 가중치 평균 값 : 1.0
8번째 맞춘 개수 : 4  / 정오표 : [1, 1, 1, 1, 0]
8번째 가중치 평균 값 : 0.933
9번째 맞춘 개수 : 5  / 정오표 : [1, 1, 1, 1, 1]
9번째 가중치 평균 값 : 1.0
10번째 맞춘 개수 : 4  / 정오표 : [1, 1, 1, 1, 0]
10번째 가중치 평균 값 : 0.933
Weighted Average Precision@K for question: 0.95


,id,img_path,overview,cat1,cat2,cat3,score,embedding_list,cat,embedding,embedding_doc2vec,doc2vec_score
16846,TRAIN_16846,./image/train/TRAIN_16846.jpg,평택 공설운동장 원불교 옆에 있으며 고기와 김치가 통으로 들어가는 김치찌개 전문점이다.,음식,음식점,한식,0.082351,"[-0.1403866410255432, 0.1987186074256897, -0.2...",한식,"[-0.14038664, 0.1987186, -0.2062287, -0.062479...","[0.10036043, -0.0927067, -0.012415466, -0.0097...",0.801289
12514,TRAIN_12514,./image/train/TRAIN_12514.jpg,횡계에서 진부를 오가는 길목에 위치한 깨끗한 숙박시설 안에 있는 가정식 백반 식당이다.,음식,음식점,한식,0.335103,"[0.2395339459180832, 0.05563323572278023, 0.14...",한식,"[0.23953395, 0.055633236, 0.1412455, 0.0472871...","[0.01555197, -0.12080012, -0.10746066, -0.0428...",0.648374
7532,TRAIN_07532,./image/train/TRAIN_07532.jpg,100% 무항생제 안동 한우만을 판매하는 곳이다. 경상북도 안동역 부근에 위치한 B...,음식,음식점,한식,0.259424,"[-0.07709348201751709, 0.3512584865093231, -0....",한식,"[-0.07709348, 0.3512585, -0.002792741, -0.1883...","[0.09773823, -0.19941016, 0.027792295, 0.00270...",0.641777
12214,TRAIN_12214,./image/train/TRAIN_12214.jpg,사장님이 직접 고른 암돼지만 취급하는 삼겹살 전문점이다.,음식,음식점,한식,0.135973,"[-0.10656668990850449, 0.3820251524448395, -0....",한식,"[-0.10656669, 0.38202515, -0.36656785, -0.3312...","[0.14205106, -0.15975623, -0.0015235783, 0.060...",0.625754
3980,TRAIN_03980,./image/train/TRAIN_03980.jpg,"경주 불국사와 석굴암에 인접하고 있는 부산식당은 한정식, 된장찌개 등 다양한 메뉴를...",음식,음식점,한식,0.277404,"[-0.35904911160469055, 0.16920025646686554, 0....",한식,"[-0.3590491, 0.16920026, 0.36173296, -0.198269...","[-0.036733903, -0.14308605, -0.029145146, 0.02...",0.623978


In [ ]:
# bert ex3 : 짧은 문장 1
true_label = '자연'
weighted_avg_precision = weighted_avg_precision(df, '낚시하기 좋은 해수욕장', true_label)
print(weighted_avg_precision)

display(bert_return_answer('낚시하기 좋은 해수욕장',df))

전체데이터 acc :  0.6
전체데이터 맞춘 개수 : 6  / 정오표 : [0, 0, 0, 1, 1, 1, 0, 1, 1, 1]
전체데이터 가중치 평균 값 : 0.436

1번째 맞춘 개수 : 6  / 정오표 : [0, 0, 0, 1, 1, 1, 0, 1, 1, 1]
1번째 가중치 평균 값 : 0.436
2번째 맞춘 개수 : 6  / 정오표 : [0, 0, 1, 0, 1, 1, 1, 1, 0, 1]
2번째 가중치 평균 값 : 0.491
3번째 맞춘 개수 : 4  / 정오표 : [0, 1, 1, 0, 1, 0, 0, 1, 0, 0]
3번째 가중치 평균 값 : 0.473
4번째 맞춘 개수 : 6  / 정오표 : [0, 0, 1, 1, 1, 1, 1, 1, 0, 0]
4번째 가중치 평균 값 : 0.6
5번째 맞춘 개수 : 6  / 정오표 : [0, 0, 0, 1, 1, 1, 1, 1, 0, 1]
5번째 가중치 평균 값 : 0.473
6번째 맞춘 개수 : 6  / 정오표 : [0, 0, 1, 0, 1, 1, 1, 1, 1, 0]
6번째 가중치 평균 값 : 0.509
7번째 맞춘 개수 : 4  / 정오표 : [0, 0, 0, 1, 1, 1, 0, 1, 0, 0]
7번째 가중치 평균 값 : 0.382
8번째 맞춘 개수 : 4  / 정오표 : [0, 1, 1, 1, 0, 0, 0, 1, 0, 0]
8번째 가중치 평균 값 : 0.491
9번째 맞춘 개수 : 5  / 정오표 : [0, 0, 0, 1, 1, 1, 1, 0, 1, 0]
9번째 가중치 평균 값 : 0.436
10번째 맞춘 개수 : 6  / 정오표 : [0, 0, 1, 1, 0, 1, 1, 1, 1, 0]
10번째 가중치 평균 값 : 0.527
Weighted Average Precision@K for question: 0.48


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
7576,7576,TRAIN_07576,./image/train/TRAIN_07576.jpg,"맑은물소리 펜션은 내린천 맑은 물에서 낚시, 래프팅을 즐기고 멱도 감을 수 있는 곳...","맑은물소리 펜션은 내린천 맑은 물에서 낚시, 래프팅을 즐기고 멱도 감을 수 있는 곳...",숙박,숙박시설,펜션,숙박,"[-0.09291666, -0.29271224, 0.28030905, 0.69830...",0.842702
9111,9111,TRAIN_09111,./image/train/TRAIN_09111.jpg,수심이 얕고 수초가 적당히 분포되어 있어 낚시에 좋은 조건을 갖추고 있다. 국도변 ...,수심이 얕고 수초가 적당히 분포되어 있어 낚시에 좋은 조건을 갖추고 있다. 국도변 ...,레포츠,수상 레포츠,민물낚시,레포츠,"[-0.48009977, -0.7611393, 0.22885725, 0.742461...",0.819359
3092,3092,TRAIN_03092,./image/train/TRAIN_03092.jpg,"저수지 인근에 오염원이 없어 물이 맑다. 수심이 얕고 수초가 적당히 분포돼 있어, ...","저수지 인근에 오염원이 없어 물이 맑다. 수심이 얕고 수초가 적당히 분포돼 있어, ...",레포츠,수상 레포츠,민물낚시,레포츠,"[-0.2524509, -0.3357086, -0.10951354, 0.635489...",0.788389
13337,13337,TRAIN_13337,./image/train/TRAIN_13337.jpg,익금해수욕장은 바다수심이 얕고 경사가 완만하며 주변에 송림숲이 잘 조성되어 있어 가...,익금해수욕장은 바다수심이 얕고 경사가 완만하며 주변에 송림숲이 잘 조성되어 있어 가...,자연,자연관광지,해수욕장,자연,"[-0.28628045, -0.2654837, 0.2721073, 0.5794812...",0.782784
13289,13289,TRAIN_13289,./image/train/TRAIN_13289.jpg,사창해수욕장은 사람들에게 많이 알려지지 않아 번잡함을 피해 휴식을 취하기 좋은 곳이...,사창해수욕장은 사람들에게 많이 알려지지 않아 번잡함을 피해 휴식을 취하기 좋은 곳이...,자연,자연관광지,해수욕장,자연,"[0.03781434, -0.2448445, 0.5636904, 0.8021647,...",0.782036
11844,11844,TRAIN_11844,./image/train/TRAIN_11844.jpg,후정해수욕장은 해안선을 따라 펼쳐진 백사장과 소나무 숲이 어우러져 시원함을 느끼게 ...,후정해수욕장은 해안선을 따라 펼쳐진 백사장과 소나무 숲이 어우러져 시원함을 느끼게 ...,자연,자연관광지,해수욕장,자연,"[-0.3130281, -0.37082803, 0.08097806, 0.743443...",0.776667
16011,16011,TRAIN_16011,./image/train/TRAIN_16011.jpg,예래생태체험축제는 '물과 함께 신명나는 해변축제'를 주제로 천혜의 경관을 자랑하는 ...,예래생태체험축제는 '물과 함께 신명나는 해변축제'를 주제로 천혜의 경관을 자랑하는 ...,인문(문화/예술/역사),축제,일반축제,축제/공연/행사,"[0.078466825, -0.34119514, 0.83666384, 0.06548...",0.765549
764,764,TRAIN_00764,./image/train/TRAIN_00764.jpg,바다를 낀 변산반도 국립공원 해안도로에서 변산과 격포를 거쳐 곰소로 가다보면 모항해...,바다를 낀 변산반도 국립공원 해안도로에서 변산과 격포를 거쳐 곰소로 가다보면 모항해...,자연,자연관광지,해수욕장,자연,"[-0.10062506, -0.65415704, -0.16887295, 0.7313...",0.765544
8106,8106,TRAIN_08106,./image/train/TRAIN_08106.jpg,삼정1리해수욕장은 구룡포해수욕장 인근에 위치하고 있다. 깨끗한 바닷물과 완만한 경사...,삼정1리해수욕장은 구룡포해수욕장 인근에 위치하고 있다. 깨끗한 바닷물과 완만한 경사...,자연,자연관광지,해수욕장,자연,"[-0.16187648, -0.26047006, 0.12439151, 0.50761...",0.763101
15033,15033,TRAIN_15033,./image/train/TRAIN_15033.jpg,다도해 해상국립공원에 위치하고 있으며 인근 자연경관이 매우 뛰어나고 환경오염이 전혀...,다도해 해상국립공원에 위치하고 있으며 인근 자연경관이 매우 뛰어나고 환경오염이 전혀...,자연,자연관광지,해수욕장,자연,"[-0.22346632, -0.03826364, 0.10012799, 0.60537...",0.760876


In [ ]:
# doc2vec ex3 : 짧은 문장 1
true_label = '자연'
query_input = '낚시하기 좋은 해수욕장'
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.0
전체데이터 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.0

1번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.0
2번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
2번째 가중치 평균 값 : 0.0
3번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
3번째 가중치 평균 값 : 0.0
4번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
4번째 가중치 평균 값 : 0.0
5번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
5번째 가중치 평균 값 : 0.0
6번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
6번째 가중치 평균 값 : 0.0
7번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.0
8번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
8번째 가중치 평균 값 : 0.0
9번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.0
10번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
10번째 가중치 평균 값 : 0.0
Weighted Average Precision@K for question: 0.0


,id,img_path,overview,cat1,cat2,cat3,score,embedding_list,cat,embedding,embedding_doc2vec,doc2vec_score
10085,TRAIN_10085,./image/train/TRAIN_10085.jpg,"설악산 울산바위 아래 콘도미니엄과 위치해 주변 경관이 빼어나고, 원암온천 지구내에 ...",레포츠,육상 레포츠,골프,0.196893,"[-0.4172859787940979, -0.023782815784215927, 1...",레포츠,"[-0.41728598, -0.023782816, 1.0406662, 0.31819...","[0.09230312, -0.14153299, 0.089427166, -0.0872...",0.752810
3431,TRAIN_03431,./image/train/TRAIN_03431.jpg,광주광역시 서구 풍암동에 위치한 아늑한 분위기의 정원이 있는 중식당이다. 30년 요...,음식,음식점,중식,0.088958,"[-0.9062281847000122, 0.18271221220493317, 0.0...",식사류,"[-0.9062282, 0.18271221, 0.07990928, -0.163321...","[0.1344617, -0.060082477, 0.04113175, -0.07270...",0.732428
12988,TRAIN_12988,./image/train/TRAIN_12988.jpg,"YC글램핑은 경기 연천군 미산면에 위치하고 있다. 부대시설로는 물놀이장, 운동시설,...",레포츠,육상 레포츠,"야영장,오토캠핑장",0.179758,"[0.012399931438267231, 0.22861598432064056, 1....",레포츠,"[0.012399931, 0.22861598, 1.0013078, 0.4909655...","[0.15203422, -0.07978523, 0.026137786, -0.1733...",0.732076
4379,TRAIN_04379,./image/train/TRAIN_04379.jpg,송어 양식장 內의 송어횟집으로 직접 생산한 송어를 대접하므로 신선하고 싱싱한 회를 ...,음식,음식점,일식,0.091580,"[-0.18444737792015076, -0.01759692095220089, -...",식사류,"[-0.18444738, -0.017596921, -0.43860963, 0.497...","[0.20104887, -0.18469015, 0.0015428684, -0.093...",0.731485
15343,TRAIN_15343,./image/train/TRAIN_15343.jpg,정갈하며 정성이 가득한 산채 정식 전문점이다. 설탕을 사용하지 않고 천연 조미료를 ...,음식,음식점,한식,0.134983,"[-0.27736079692840576, 0.8594754338264465, -0....",한식,"[-0.2773608, 0.85947543, -0.26301122, -0.02724...","[0.20580965, -0.10907373, 0.011237411, -0.1036...",0.729509


In [ ]:
# bert ex4 : 짧은 문장 2
true_label = '역사관광지'
weighted_avg_precision = weighted_avg_precision(df, '유명한 절', true_label)
print(weighted_avg_precision)

display(bert_return_answer('유명한 절',df))

전체데이터 acc :  0.1
전체데이터 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.091

1번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.127
2번째 맞춘 개수 : 2  / 정오표 : [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
2번째 가중치 평균 값 : 0.145
3번째 맞춘 개수 : 2  / 정오표 : [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
3번째 가중치 평균 값 : 0.145
4번째 맞춘 개수 : 2  / 정오표 : [0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
4번째 가중치 평균 값 : 0.127
5번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
5번째 가중치 평균 값 : 0.055
6번째 맞춘 개수 : 2  / 정오표 : [0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
6번째 가중치 평균 값 : 0.127
7번째 맞춘 개수 : 2  / 정오표 : [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
7번째 가중치 평균 값 : 0.145
8번째 맞춘 개수 : 2  / 정오표 : [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
8번째 가중치 평균 값 : 0.164
9번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
9번째 가중치 평균 값 : 0.073
10번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
10번째 가중치 평균 값 : 0.109
Weighted Average Precision@K for question: 0.12


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
16208,16208,TRAIN_16208,./image/train/TRAIN_16208.jpg,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.<br>,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.,음식,음식점,한식,한식,"[-0.065323845, -0.07872713, 0.74007475, -0.019...",0.798621
2570,2570,TRAIN_02570,./image/train/TRAIN_02570.jpg,동해시 천곡동에 위치하였으며 도루묵 전문식당으로 유명하다.,동해시 천곡동에 위치하였으며 도루묵 전문식당으로 유명하다.,음식,음식점,한식,한식,"[-0.05982764, 0.17517398, 0.53054136, -0.15591...",0.667726
3030,3030,TRAIN_03030,./image/train/TRAIN_03030.jpg,홈페이지 참조,홈페이지 참조,인문(문화/예술/역사),공연/행사,기타행사,축제/공연/행사,"[0.2027652, 0.14053588, 0.6965719, -0.04203199...",0.661382
3387,3387,TRAIN_03387,./image/train/TRAIN_03387.jpg,산모랭이펜션은 진부면에 위치하고 있다.,산모랭이펜션은 진부면에 위치하고 있다.,숙박,숙박시설,펜션,숙박,"[-0.068702996, -0.08789411, 0.81154406, 0.0506...",0.647767
3909,3909,TRAIN_03909,./image/train/TRAIN_03909.jpg,"반려견카페로 반려견 없이 입장가능, 반려견 입장료 3,000원을 받음","반려견카페로 반려견 없이 입장가능, 반려견 입장료 3,000원을 받음",음식,음식점,바/까페,바/까페,"[-0.10746452, 0.11642189, 0.6907376, 0.0520122...",0.630078
13826,13826,TRAIN_13826,./image/train/TRAIN_13826.jpg,"* 초의 장의순 대선사를 기리는 곳, 초의선사 유적지 *<br /><br />초의(...","* 초의 장의순 대선사를 기리는 곳, 초의선사 유적지 *초의(장의순) 대선사의 출생...",인문(문화/예술/역사),역사관광지,유적지/사적지,역사관광지,"[0.055031225, 0.5522561, 0.7372338, -0.1034661...",0.591831
2470,2470,TRAIN_02470,./image/train/TRAIN_02470.jpg,전통과 현대가 어우러진 고즈넉한 한옥 숙소이다.,전통과 현대가 어우러진 고즈넉한 한옥 숙소이다.,숙박,숙박시설,홈스테이,숙박,"[-0.08771517, -0.0821382, 0.98729205, 0.285370...",0.569285
12699,12699,TRAIN_12699,./image/train/TRAIN_12699.jpg,환타피아는 용평리조트에 위치하고 있는 레포츠 샵으로서 여유로운 레져 문화의 선두주자...,환타피아는 용평리조트에 위치하고 있는 레포츠 샵으로서 여유로운 레져 문화의 선두주자...,레포츠,육상 레포츠,스키(보드) 렌탈샵,레포츠,"[-0.015702706, 0.10240584, 0.86192626, -0.1025...",0.568210
11168,11168,TRAIN_11168,./image/train/TRAIN_11168.jpg,역사의 깊이가 있는 고택이다.,역사의 깊이가 있는 고택이다.,숙박,숙박시설,홈스테이,숙박,"[0.71944225, 0.42473668, 1.2885308, -0.1919653...",0.566647
4277,4277,TRAIN_04277,./image/train/TRAIN_04277.jpg,"섬의 형상이 발가락, 거북이을 닮은 형상을 하고 있으며 이곳에 낙조가 떨어질 때면 ...","섬의 형상이 발가락, 거북이을 닮은 형상을 하고 있으며 이곳에 낙조가 떨어질 때면 ...",자연,자연관광지,섬,자연,"[0.5462943, -0.39851886, 0.9997766, 0.10550959...",0.560716


In [ ]:
# doc2vec ex4 : 짧은 문장 2
true_label = '역사관광지'
query_input = '유명한 절'
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.0
전체데이터 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.0

1번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.0
2번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
2번째 가중치 평균 값 : 0.0
3번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
3번째 가중치 평균 값 : 0.0
4번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
4번째 가중치 평균 값 : 0.0
5번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
5번째 가중치 평균 값 : 0.0
6번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
6번째 가중치 평균 값 : 0.0
7번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.0
8번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
8번째 가중치 평균 값 : 0.0
9번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.0
10번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
10번째 가중치 평균 값 : 0.0
Weighted Average Precision@K for question: 0.0


,id,img_path,overview,cat1,cat2,cat3,score,embedding_list,cat,embedding,embedding_doc2vec,doc2vec_score
11153,TRAIN_11153,./image/train/TRAIN_11153.jpg,현지인들이 가는 숨은 맛집이다. 대표메뉴는 동태탕이다. 전라남도 순천시에 있는 한식...,음식,음식점,한식,0.119112,"[-0.5453720092773438, 0.33767372369766235, 0.1...",한식,"[-0.545372, 0.33767372, 0.18319285, 0.14593175...","[0.11471534, -0.034332193, -0.009249231, -0.06...",0.713636
12988,TRAIN_12988,./image/train/TRAIN_12988.jpg,"YC글램핑은 경기 연천군 미산면에 위치하고 있다. 부대시설로는 물놀이장, 운동시설,...",레포츠,육상 레포츠,"야영장,오토캠핑장",0.261357,"[0.012399931438267231, 0.22861598432064056, 1....",레포츠,"[0.012399931, 0.22861598, 1.0013078, 0.4909655...","[0.15203422, -0.07978523, 0.026137786, -0.1733...",0.712660
7607,TRAIN_07607,./image/train/TRAIN_07607.jpg,남선옥 식육식당은 국내산 순수 한우만을 취급하는 50년 넘는 전통의 한우 숯불구이 ...,음식,음식점,한식,0.116291,"[-0.38297238945961, -0.05408414453268051, -0.0...",한식,"[-0.3829724, -0.054084145, -0.022685014, -0.23...","[0.10019222, -0.08465039, -0.004923448, -0.070...",0.711560
6661,TRAIN_06661,./image/train/TRAIN_06661.jpg,"동해안 자연산 해산물 전문점, 강원도에서 나오는 제철 해산물을 판매한다.",음식,음식점,한식,0.157898,"[-0.3752511739730835, 0.34668973088264465, -0....",한식,"[-0.37525117, 0.34668973, -0.41285047, 0.09936...","[0.11860011, -0.22290601, 0.061102133, -0.0876...",0.708368
9908,TRAIN_09908,./image/train/TRAIN_09908.jpg,한옥마을 중심에 위치한 80년 전통 한옥이다.,숙박,숙박시설,홈스테이,0.411624,"[0.2736412584781647, 0.25149834156036377, 0.65...",숙박,"[0.27364126, 0.25149834, 0.6514545, -0.528978,...","[0.15147029, -0.07863583, -0.029420387, -0.065...",0.702769


In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def bert_return_answer(question, df):
  embedding = sbert_model.encode(question)
  df['score'] = df.apply(lambda x: cos_sim(x['SBERT_emb'], embedding), axis=1)
  return df.loc[df['score'].nlargest(10).index]

def weighted_avg_precision(df, question, true_label, k=10, weights=None):
    total_precision = 0

    if weights is None:
        # weights = [5/15, 4/15, 3/15, 2/15, 1/15]
        weights = [10/55, 9/55, 8/55, 7/55, 6/55, 5/55, 4/55, 3/55, 2/55, 1/55]

    total_res = bert_return_answer(question, df)
    tot_pred_labels = total_res['cat'].tolist()
    tot_correct_scores = [1 if tot_pred_labels[i] == true_label else 0 for i in range(len(tot_pred_labels))]
    tot_precision = sum([tot_correct_scores[i] * weights[i] for i in range(len(tot_pred_labels))])
    print('전체데이터 acc : ', sum(tot_correct_scores) / 10)
    print(f'전체데이터 맞춘 개수 : {sum(tot_correct_scores)}', f' / 정오표 : {tot_correct_scores}')
    print(f'전체데이터 가중치 평균 값 : {round(tot_precision, 3)}')
    print()

    for i in range(k):
        # 샘플링 0.7 비율로 k 번 진행
        sss = StratifiedShuffleSplit(n_splits=11, test_size=0.7)
        for train_index, test_index in sss.split(df, df['cat']):
          sample_df = df.iloc[test_index]

        result = bert_return_answer(question, sample_df)
        pred_labels = result['cat'].tolist()

        correct_scores = [1 if pred_labels[i] == true_label else 0 for i in range(len(pred_labels))]
        precision_at_k = sum([correct_scores[i] * weights[i] for i in range(len(pred_labels))])
        total_precision += precision_at_k

        print(f'{i+1}번째 맞춘 개수 : {sum(correct_scores)}', f' / 정오표 : {correct_scores}')
        print(f'{i+1}번째 가중치 평균 값 : {round(precision_at_k,3)}')

        res = "Weighted Average Precision@K for question: " + str(round(total_precision / k, 2))
    return res

# bert ex5 : 문서 전체
true_label = '바/까페'
input_query = df['overview'][8589]
bert_return_answer(input_query,df)
res = weighted_avg_precision(df, input_query, true_label,k=10)
print(res)
display(bert_return_answer(input_query,df))


전체데이터 acc :  0.8
전체데이터 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
전체데이터 가중치 평균 값 : 0.873

1번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
1번째 가중치 평균 값 : 0.818
2번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
2번째 가중치 평균 값 : 0.873
3번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]
3번째 가중치 평균 값 : 0.855
4번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
4번째 가중치 평균 값 : 0.945
5번째 맞춘 개수 : 7  / 정오표 : [1, 1, 0, 1, 1, 0, 1, 1, 1, 0]
5번째 가중치 평균 값 : 0.745
6번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 0, 0, 1, 1, 1, 1, 1]
6번째 가중치 평균 값 : 0.764
7번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
7번째 가중치 평균 값 : 0.873
8번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
8번째 가중치 평균 값 : 0.909
9번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
9번째 가중치 평균 값 : 0.927
10번째 맞춘 개수 : 7  / 정오표 : [1, 1, 1, 0, 1, 1, 1, 0, 1, 0]
10번째 가중치 평균 값 : 0.8
Weighted Average Precision@K for question: 0.85


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
8589,8589,TRAIN_08589,./image/train/TRAIN_08589.jpg,"갓볶은 신선한 원두로 내린 커피와 좋은 재료로 만든 수제마카롱이 있으며, 여러 음료...","갓볶은 신선한 원두로 내린 커피와 좋은 재료로 만든 수제마카롱이 있으며, 여러 음료...",음식,음식점,바/까페,바/까페,"[-0.19067353, 0.2397516, 0.4691922, 0.04981446...",1.000000
7737,7737,TRAIN_07737,./image/train/TRAIN_07737.jpg,"경기도 광주시 오포읍에 있는 베이커리 카페로 커피, 라테, 스무디, 에이드, 베이커...","경기도 광주시 오포읍에 있는 베이커리 카페로 커피, 라테, 스무디, 에이드, 베이커...",음식,음식점,바/까페,바/까페,"[-0.7410138, 0.6095806, -0.15320836, -0.034829...",0.854282
6446,6446,TRAIN_06446,./image/train/TRAIN_06446.jpg,아눅앞산은 대구시 남구 대명동에 자리 잡고 있다. 대구 시내를 조망할 수 있는 위치...,아눅앞산은 대구시 남구 대명동에 자리 잡고 있다. 대구 시내를 조망할 수 있는 위치...,음식,음식점,바/까페,바/까페,"[-0.7217448, 0.12427751, 0.2755149, -0.0528174...",0.846402
5975,5975,TRAIN_05975,./image/train/TRAIN_05975.jpg,카페화담은 경북 구미시 오태동에 있다.<br>대표 메뉴는 아메리카노다. 이 밖에 바...,카페화담은 경북 구미시 오태동에 있다.대표 메뉴는 아메리카노다. 이 밖에 바닐라라뗴...,음식,음식점,바/까페,바/까페,"[-0.49524435, 0.052694052, 0.44811672, -0.1388...",0.838855
8770,8770,TRAIN_08770,./image/train/TRAIN_08770.jpg,돈가스와메밀 광화문미진은 세종시 고운동에 자리 잡고 있다. 대표 메뉴는 판 메밀이다...,돈가스와메밀 광화문미진은 세종시 고운동에 자리 잡고 있다. 대표 메뉴는 판 메밀이다...,음식,음식점,한식,한식,"[-0.6126176, 0.16511567, 0.2580007, 0.13142146...",0.838599
4852,4852,TRAIN_04852,./image/train/TRAIN_04852.jpg,"빌리웍스는 베이커리와 카페, 전시와 공연이 공존하는 문화예술 복합공간이다. 대표 메...","빌리웍스는 베이커리와 카페, 전시와 공연이 공존하는 문화예술 복합공간이다. 대표 메...",음식,음식점,바/까페,바/까페,"[-0.5418691, 0.43519688, 0.42377275, -0.398400...",0.837645
2861,2861,TRAIN_02861,./image/train/TRAIN_02861.jpg,아우트로 커피 산성은 충청북도 청주시 상당구 낭성면에 자리 잡은 대형 카페이다. 상...,아우트로 커피 산성은 충청북도 청주시 상당구 낭성면에 자리 잡은 대형 카페이다. 상...,음식,음식점,바/까페,바/까페,"[-0.80520964, 0.40175745, 0.2720502, -0.052851...",0.835560
1486,1486,TRAIN_01486,./image/train/TRAIN_01486.jpg,앙로고택은 인간문화재가 건축한 한옥카페이다. 본관과 별관으로 나누어져 있으며 음료와...,앙로고택은 인간문화재가 건축한 한옥카페이다. 본관과 별관으로 나누어져 있으며 음료와...,음식,음식점,바/까페,바/까페,"[-0.47512043, 0.7497827, 0.12899187, -0.274602...",0.826939
1668,1668,TRAIN_01668,./image/train/TRAIN_01668.jpg,"쎈느(Scene)는 2호선 성수역 4번 출구 성수동 구두테마공원 옆, 성수동 카페거...","쎈느(Scene)는 2호선 성수역 4번 출구 성수동 구두테마공원 옆, 성수동 카페거...",음식,음식점,바/까페,바/까페,"[-0.26352876, 0.5089336, 0.2323926, -0.1382180...",0.826937
41,41,TRAIN_00041,./image/train/TRAIN_00041.jpg,신선한 회와 다양한 스끼다시의 세계를 맛볼 수 있는 노량진수산물도매시장을 대표하는 ...,신선한 회와 다양한 스끼다시의 세계를 맛볼 수 있는 노량진수산물도매시장을 대표하는 ...,음식,음식점,한식,한식,"[-0.31323335, 0.40946946, 0.03088137, -0.06375...",0.826018


In [ ]:
# doc2vec ex5 : 문서 전체
true_label = '레포츠'
query_input = df['overview'][1]
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.4
전체데이터 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.6

1번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
1번째 가중치 평균 값 : 0.6
2번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
2번째 가중치 평균 값 : 0.6
3번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
3번째 가중치 평균 값 : 0.6
4번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
4번째 가중치 평균 값 : 0.333
5번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
5번째 가중치 평균 값 : 0.6
6번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
6번째 가중치 평균 값 : 0.6
7번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.333
8번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
8번째 가중치 평균 값 : 0.6
9번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
9번째 가중치 평균 값 : 0.6
10번째 맞춘 개수 : 2  / 정오표 : [1, 1, 0, 0, 0]
10번째 가중치 평균 값 : 0.6
Weighted Average Precision@K for question: 0.55


,id,img_path,overview,cat1,cat2,cat3,score,embedding_list,cat,embedding,embedding_doc2vec,doc2vec_score
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,0.446187,"[-0.4031026065349579, 0.36999034881591797, 0.4...",레포츠,"[-0.4031026, 0.36999035, 0.46873328, 0.0787834...","[0.088259056, -0.43449298, -0.08759773, 0.1295...",0.962263
8036,TRAIN_08036,./image/train/TRAIN_08036.jpg,"경기도 이천시 설성면에 위치한 골프장으로 대증제 27홀이다. 약 1,400,000m...",레포츠,육상 레포츠,골프,0.317470,"[-0.3489031195640564, 0.1818375587463379, 0.24...",레포츠,"[-0.34890312, 0.18183756, 0.24129055, 0.012580...","[0.18973836, -0.8272857, 0.050687734, -0.31743...",0.641568
5276,TRAIN_05276,./image/train/TRAIN_05276.jpg,"국내외전통, 세계,남북음식문화를소개하는 행사이다.\n전통음식모든 것을 한자리에 볼 ...",인문(문화/예술/역사),공연/행사,기타행사,0.144593,"[-0.28302767872810364, -0.08312062919139862, 1...",축제/공연/행사,"[-0.28302768, -0.08312063, 1.1806757, 0.035646...","[0.11373998, -0.22405417, 0.010375781, 0.06329...",0.537727
1887,TRAIN_01887,./image/train/TRAIN_01887.jpg,"통영은 고성반도 끝자락과 미륵도, 한산도를 비롯한 부속 섬 40여개로 이루어진 고장...",인문(문화/예술/역사),역사관광지,성,0.294103,"[0.13529960811138153, 0.2723046541213989, 0.59...",역사관광지,"[0.13529961, 0.27230465, 0.5975502, 0.18918002...","[-0.20470376, -0.9693397, 0.40068266, 0.420306...",0.532826
8122,TRAIN_08122,./image/train/TRAIN_08122.jpg,대구시 북구 산격동에는 숨은 볼거리가 많다. 이 가운데 하나가 연암공원이다. 연암공...,인문(문화/예술/역사),휴양관광지,공원,0.110787,"[-0.15069548785686493, 0.039623185992240906, 0...",휴양/체험/산업관광지,"[-0.15069549, 0.039623186, 0.31503767, 0.28261...","[0.034397416, -0.47131625, -0.3108373, 0.11277...",0.520340


In [ ]:
true_label = '레포츠'
input_query = '골프'
bert_return_answer(input_query,df)
res = weighted_avg_precision(df, input_query, true_label,k=10)
print(res)
display(bert_return_answer(input_query,df))

전체데이터 acc :  1.0
전체데이터 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
전체데이터 가중치 평균 값 : 1.0

1번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
1번째 가중치 평균 값 : 0.964
2번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2번째 가중치 평균 값 : 1.0
3번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]
3번째 가중치 평균 값 : 0.855
4번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
4번째 가중치 평균 값 : 1.0
5번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
5번째 가중치 평균 값 : 0.945
6번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
6번째 가중치 평균 값 : 0.964
7번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
7번째 가중치 평균 값 : 0.964
8번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
8번째 가중치 평균 값 : 0.945
9번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
9번째 가중치 평균 값 : 0.964
10번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
10번째 가중치 평균 값 : 0.945
Weighted Average Precision@K for question: 0.95


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
1811,1811,TRAIN_01811,./image/train/TRAIN_01811.jpg,SG아름다운골프앤리조트는 충남 아산시 영인면에 자리 잡고 있다.\n골프장은 영인산 ...,SG아름다운골프앤리조트는 충남 아산시 영인면에 자리 잡고 있다.골프장은 영인산 자락...,레포츠,육상 레포츠,골프,레포츠,"[-0.54339015, 0.36494535, 0.86161417, -0.39267...",0.589865
13946,13946,TRAIN_13946,./image/train/TRAIN_13946.jpg,아리스타 CC는 충남 논산시 연무읍 황화정리에 자리 잡고 있다. 이곳은 대중제 골프...,아리스타 CC는 충남 논산시 연무읍 황화정리에 자리 잡고 있다. 이곳은 대중제 골프...,레포츠,육상 레포츠,골프,레포츠,"[0.2576525, 0.14814349, 1.0360498, 0.057228796...",0.569130
3453,3453,TRAIN_03453,./image/train/TRAIN_03453.jpg,인천 중구 운서동에 위치한 스카이72GC 바다코스는 스카이72에 소속된 4가지 코스...,인천 중구 운서동에 위치한 스카이72GC 바다코스는 스카이72에 소속된 4가지 코스...,레포츠,육상 레포츠,골프,레포츠,"[-0.6000029, 0.16667673, 0.9276301, -0.0235539...",0.568106
8332,8332,TRAIN_08332,./image/train/TRAIN_08332.jpg,우정힐스CC는 충남 천안시 동남구 목천읍에 자리 잡고 있다.\n이곳은 회원제 골프장...,우정힐스CC는 충남 천안시 동남구 목천읍에 자리 잡고 있다.이곳은 회원제 골프장으로...,레포츠,육상 레포츠,골프,레포츠,"[-0.09546193, 0.3451574, 1.1621594, -0.1899070...",0.561532
12929,12929,TRAIN_12929,./image/train/TRAIN_12929.jpg,제주경마공원에서 제주테마파크 조성사업의 일환으로 도내 최대규모의 파크골프장을 개장하...,제주경마공원에서 제주테마파크 조성사업의 일환으로 도내 최대규모의 파크골프장을 개장하...,레포츠,육상 레포츠,골프,레포츠,"[-0.6660219, -0.19458151, 0.70935863, -0.04128...",0.554573
6703,6703,TRAIN_06703,./image/train/TRAIN_06703.jpg,레이캐슬 CC는 세종시 전의면 달전리에 자리 잡고 있다. 이곳은 대중제 골프장으로 ...,레이캐슬 CC는 세종시 전의면 달전리에 자리 잡고 있다. 이곳은 대중제 골프장으로 ...,레포츠,육상 레포츠,골프,레포츠,"[-0.2669283, 0.44213396, 0.65915984, -0.179789...",0.552714
10986,10986,TRAIN_10986,./image/train/TRAIN_10986.jpg,경기도 이천시 장호원읍 풍계리에 있는 골프장으로 회원제 18홀과 대중제 9홀이 있다...,경기도 이천시 장호원읍 풍계리에 있는 골프장으로 회원제 18홀과 대중제 9홀이 있다...,레포츠,육상 레포츠,골프,레포츠,"[-0.4824579, -0.2706711, 0.9936754, 0.23371856...",0.532360
12319,12319,TRAIN_12319,./image/train/TRAIN_12319.jpg,백제CC는 충남 부여군 은산면 나령리에 자리 잡고 있다.\n이곳은 대중제 골프장으로...,백제CC는 충남 부여군 은산면 나령리에 자리 잡고 있다.이곳은 대중제 골프장으로 2...,레포츠,육상 레포츠,골프,레포츠,"[-0.012670167, 0.5528972, 0.8561883, -0.327994...",0.531088
2122,2122,TRAIN_02122,./image/train/TRAIN_02122.jpg,파인스톤CC는 충남 당진시 송산면 무수리에 자리 잡고 있다. 이곳은 대중제 골프장으...,파인스톤CC는 충남 당진시 송산면 무수리에 자리 잡고 있다. 이곳은 대중제 골프장으...,레포츠,육상 레포츠,골프,레포츠,"[0.1097219, 0.113096826, 0.9749177, -0.0610404...",0.527271
14022,14022,TRAIN_14022,./image/train/TRAIN_14022.jpg,경기도 고양시 벽제의 수려한 자연경관속에 위치한 올림픽C.C는 강북권에서 15~30...,경기도 고양시 벽제의 수려한 자연경관속에 위치한 올림픽C.C는 강북권에서 15~30...,레포츠,육상 레포츠,골프,레포츠,"[-0.80776733, -0.13848884, 0.55610555, 0.17152...",0.519864


In [ ]:
# doc2vec ex5 : 문서 전체
true_label = '레포츠'
query_input = '골프'
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.2
전체데이터 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.333

1번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.333
2번째 맞춘 개수 : 1  / 정오표 : [0, 0, 1, 0, 0]
2번째 가중치 평균 값 : 0.2
3번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
3번째 가중치 평균 값 : 0.333
4번째 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 0, 1]
4번째 가중치 평균 값 : 0.333
5번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 1, 0]
5번째 가중치 평균 값 : 0.133
6번째 맞춘 개수 : 2  / 정오표 : [1, 0, 0, 1, 0]
6번째 가중치 평균 값 : 0.467
7번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.333
8번째 맞춘 개수 : 2  / 정오표 : [1, 0, 1, 0, 0]
8번째 가중치 평균 값 : 0.533
9번째 맞춘 개수 : 1  / 정오표 : [1, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.333
10번째 맞춘 개수 : 2  / 정오표 : [0, 0, 0, 1, 1]
10번째 가중치 평균 값 : 0.2
Weighted Average Precision@K for question: 0.32


,id,img_path,overview,cat1,cat2,cat3,score,embedding_list,cat,embedding,embedding_doc2vec,doc2vec_score
14224,TRAIN_14224,./image/train/TRAIN_14224.jpg,"과학과 관광이 어우러진 대전 유성에서 터를 잡은지 30여년, 골프 스포츠로서 인정되...",레포츠,육상 레포츠,골프,0.435806,"[-0.38147905468940735, 0.11000160127878189, 0....",레포츠,"[-0.38147905, 0.1100016, 0.5477136, -0.4652805...","[0.25052655, -0.11942878, 0.14811386, 0.156858...",0.897536
15540,TRAIN_15540,./image/train/TRAIN_15540.jpg,현지인이 추천하는 모범음식점이다. 대표메뉴는 쌈밥이다. 전라남도 목포시에 있는 한식...,음식,음식점,한식,0.146352,"[-0.08069417625665665, 0.3989069163799286, 0.1...",한식,"[-0.080694176, 0.39890692, 0.10864239, -0.1844...","[0.12884112, -0.060911436, -0.00644402, -0.029...",0.700028
15343,TRAIN_15343,./image/train/TRAIN_15343.jpg,정갈하며 정성이 가득한 산채 정식 전문점이다. 설탕을 사용하지 않고 천연 조미료를 ...,음식,음식점,한식,0.032734,"[-0.27736079692840576, 0.8594754338264465, -0....",한식,"[-0.2773608, 0.85947543, -0.26301122, -0.02724...","[0.20580965, -0.10907373, 0.011237411, -0.1036...",0.695035
11115,TRAIN_11115,./image/train/TRAIN_11115.jpg,월성식당은 강릉에서 장치찜으로 유명한 음식점이다. 이름조차 생소한 장치는 동해안의 ...,음식,음식점,한식,0.002333,"[-0.48108962178230286, -0.13686347007751465, 0...",한식,"[-0.48108962, -0.13686347, 0.21637072, -0.0172...","[0.1773543, -0.1926902, -0.07286622, -0.069639...",0.694479
16261,TRAIN_16261,./image/train/TRAIN_16261.jpg,봉평면에 위치하고 있는 봉평메밀닭강정은 현지인 추천 맛집이다. 메밀을 섞어 만든 닭...,음식,음식점,한식,-0.062052,"[-0.003681828500702977, -0.07224772125482559, ...",한식,"[-0.0036818285, -0.07224772, -0.8921418, 0.509...","[0.15673661, -0.15800638, -0.02909074, -0.0864...",0.687895


In [ ]:
true_label = '레포츠'
input_query = '골프치기 좋은 곳'
bert_return_answer(input_query,df)
res = weighted_avg_precision(df, input_query, true_label,k=10)
print(res)
display(bert_return_answer(input_query,df))

전체데이터 acc :  0.9
전체데이터 맞춘 개수 : 9  / 정오표 : [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
전체데이터 가중치 평균 값 : 0.818

1번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
1번째 가중치 평균 값 : 0.927
2번째 맞춘 개수 : 8  / 정오표 : [0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
2번째 가중치 평균 값 : 0.764
3번째 맞춘 개수 : 7  / 정오표 : [0, 1, 1, 1, 0, 1, 1, 1, 0, 1]
3번째 가중치 평균 값 : 0.673
4번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
4번째 가중치 평균 값 : 1.0
5번째 맞춘 개수 : 8  / 정오표 : [0, 1, 1, 1, 1, 0, 1, 1, 1, 1]
5번째 가중치 평균 값 : 0.727
6번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
6번째 가중치 평균 값 : 0.982
7번째 맞춘 개수 : 8  / 정오표 : [0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
7번째 가중치 평균 값 : 0.782
8번째 맞춘 개수 : 8  / 정오표 : [0, 1, 1, 1, 1, 1, 0, 1, 1, 1]
8번째 가중치 평균 값 : 0.745
9번째 맞춘 개수 : 8  / 정오표 : [0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
9번째 가중치 평균 값 : 0.764
10번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
10번째 가중치 평균 값 : 1.0
Weighted Average Precision@K for question: 0.84


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
15664,15664,TRAIN_15664,./image/train/TRAIN_15664.jpg,설악산과 금강산이 이어지는 그림 같은 산자락에 펼쳐진 설악썬밸리골프리조트는 앞으로는...,설악산과 금강산이 이어지는 그림 같은 산자락에 펼쳐진 설악썬밸리골프리조트는 앞으로는...,인문(문화/예술/역사),휴양관광지,관광단지,휴양/체험/산업관광지,"[-0.5146854, -0.07017649, 1.0551479, 0.0816385...",0.701647
7724,7724,TRAIN_07724,./image/train/TRAIN_07724.jpg,올데이 골프앤리조트는 충북 충주시 앙성면 돈산리에 자리 잡고 있다. 이곳은 27홀로...,올데이 골프앤리조트는 충북 충주시 앙성면 돈산리에 자리 잡고 있다. 이곳은 27홀로...,레포츠,육상 레포츠,골프,레포츠,"[-0.6957091, 0.31341314, 0.7358337, -0.0868858...",0.685676
3957,3957,TRAIN_03957,./image/train/TRAIN_03957.jpg,경기도 여주시에 위치해 있는 퍼블릭 골프장이다. 남한강 인근에 자리하고 있으며 여주...,경기도 여주시에 위치해 있는 퍼블릭 골프장이다. 남한강 인근에 자리하고 있으며 여주...,레포츠,육상 레포츠,골프,레포츠,"[-0.7256857, -0.08133174, 0.48878455, 0.086004...",0.681394
14799,14799,TRAIN_14799,./image/train/TRAIN_14799.jpg,파인리즈컨트리클럽은 남한의 금강산으로 일컬어지는 설악산을 등에 지고 장엄한 생명의 ...,파인리즈컨트리클럽은 남한의 금강산으로 일컬어지는 설악산을 등에 지고 장엄한 생명의 ...,레포츠,육상 레포츠,골프,레포츠,"[-0.89527935, -0.23389591, 0.7792163, 0.167582...",0.669999
14066,14066,TRAIN_14066,./image/train/TRAIN_14066.jpg,서원은 상서로운 장소와 복되고 길한 일이 일어나는 장소를 의미하는 경기도 파주의 옛...,서원은 상서로운 장소와 복되고 길한 일이 일어나는 장소를 의미하는 경기도 파주의 옛...,레포츠,육상 레포츠,골프,레포츠,"[-0.83283615, -0.064237006, 0.75223, 0.3772300...",0.663639
8126,8126,TRAIN_08126,./image/train/TRAIN_08126.jpg,타이거컨트리클럽은 당동IC에서 식현사거리를 거쳐 타이거CC까지 17.5km 거리에 ...,타이거컨트리클럽은 당동IC에서 식현사거리를 거쳐 타이거CC까지 17.5km 거리에 ...,레포츠,육상 레포츠,골프,레포츠,"[-0.2984272, 0.06684689, 0.67840856, -0.061442...",0.654469
5368,5368,TRAIN_05368,./image/train/TRAIN_05368.jpg,남녀노소 누구나 부담없이 즐길 수 있는 회원제 골프장이다\n골프 문화의 대중화와 정...,"남녀노소 누구나 부담없이 즐길 수 있는 회원제 골프장이다골프 문화의 대중화와 정착,...",레포츠,육상 레포츠,골프,레포츠,"[-0.2585938, 0.24609783, 0.68445396, 0.0197528...",0.645254
13766,13766,TRAIN_13766,./image/train/TRAIN_13766.jpg,"태광 컨트리클럽은 쾌적한 환경, 천혜적인 지리적 위치, 편리한 교통으로 골퍼들의 사...","태광 컨트리클럽은 쾌적한 환경, 천혜적인 지리적 위치, 편리한 교통으로 골퍼들의 사...",레포츠,육상 레포츠,골프,레포츠,"[-0.3050921, -0.06903075, 0.6712569, -0.127257...",0.643454
1811,1811,TRAIN_01811,./image/train/TRAIN_01811.jpg,SG아름다운골프앤리조트는 충남 아산시 영인면에 자리 잡고 있다.\n골프장은 영인산 ...,SG아름다운골프앤리조트는 충남 아산시 영인면에 자리 잡고 있다.골프장은 영인산 자락...,레포츠,육상 레포츠,골프,레포츠,"[-0.54339015, 0.36494535, 0.86161417, -0.39267...",0.639630
14208,14208,TRAIN_14208,./image/train/TRAIN_14208.jpg,공무원연금공단이 운영하는 김해 상록골프장은 김해에 위치한 정규 18홀로 구성된 대중...,공무원연금공단이 운영하는 김해 상록골프장은 김해에 위치한 정규 18홀로 구성된 대중...,레포츠,육상 레포츠,골프,레포츠,"[-0.48843348, -0.08937794, 0.52585787, -0.0146...",0.634064


In [ ]:
# doc2vec
true_label = '레포츠'
query_input = '골프치기 좋은 곳'
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.0
전체데이터 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.0

1번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 0, 1]
1번째 가중치 평균 값 : 0.067
2번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
2번째 가중치 평균 값 : 0.0
3번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
3번째 가중치 평균 값 : 0.0
4번째 맞춘 개수 : 1  / 정오표 : [0, 0, 1, 0, 0]
4번째 가중치 평균 값 : 0.2
5번째 맞춘 개수 : 1  / 정오표 : [0, 0, 1, 0, 0]
5번째 가중치 평균 값 : 0.2
6번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
6번째 가중치 평균 값 : 0.0
7번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.0
8번째 맞춘 개수 : 1  / 정오표 : [0, 1, 0, 0, 0]
8번째 가중치 평균 값 : 0.267
9번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.0
10번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
10번째 가중치 평균 값 : 0.0
Weighted Average Precision@K for question: 0.07


,id,img_path,overview,cat1,cat2,cat3,score,embedding_list,cat,embedding,embedding_doc2vec,doc2vec_score
7582,TRAIN_07582,./image/train/TRAIN_07582.jpg,성남시에서 운영하고 있는 은행식물원은 성남시 중원구 은행1동에 양묘장과 용도폐기된 ...,자연,자연관광지,수목원,0.245697,"[-0.5613351464271545, 0.14166174829006195, 0.1...",자연,"[-0.56133515, 0.14166175, 0.14076936, 0.278045...","[-0.18418777, -0.1231371, 0.24571103, 0.169290...",0.793760
14085,TRAIN_14085,./image/train/TRAIN_14085.jpg,전주시 교동 한옥마을에 자리잡고 있는 다문(茶門)'은 한옥 건물이 둘러싸고 있는 식...,음식,음식점,한식,0.292926,"[0.19401301443576813, 0.3904130160808563, 0.44...",한식,"[0.19401301, 0.39041302, 0.4416429, -0.2825594...","[0.1671088, -0.09970581, -0.023393642, -0.0618...",0.776866
3431,TRAIN_03431,./image/train/TRAIN_03431.jpg,광주광역시 서구 풍암동에 위치한 아늑한 분위기의 정원이 있는 중식당이다. 30년 요...,음식,음식점,중식,0.191716,"[-0.9062281847000122, 0.18271221220493317, 0.0...",식사류,"[-0.9062282, 0.18271221, 0.07990928, -0.163321...","[0.1344617, -0.060082477, 0.04113175, -0.07270...",0.774972
3030,TRAIN_03030,./image/train/TRAIN_03030.jpg,홈페이지 참조,인문(문화/예술/역사),공연/행사,기타행사,0.473727,"[0.202765092253685, 0.1405358612537384, 0.6965...",축제/공연/행사,"[0.20276509, 0.14053586, 0.6965728, -0.0420316...","[0.13619375, -0.09631479, -0.049492933, -0.055...",0.770224
12988,TRAIN_12988,./image/train/TRAIN_12988.jpg,"YC글램핑은 경기 연천군 미산면에 위치하고 있다. 부대시설로는 물놀이장, 운동시설,...",레포츠,육상 레포츠,"야영장,오토캠핑장",0.257523,"[0.012399931438267231, 0.22861598432064056, 1....",레포츠,"[0.012399931, 0.22861598, 1.0013078, 0.4909655...","[0.15203422, -0.07978523, 0.026137786, -0.1733...",0.769741


### test2

In [ ]:
true_label = '바/까페'
query_input = '커피'
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

# doc2vec
# res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
# print(res)
# display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.7
전체데이터 맞춘 개수 : 7  / 정오표 : [0, 1, 1, 1, 1, 1, 0, 1, 1, 0]
전체데이터 가중치 평균 값 : 0.727

1번째 맞춘 개수 : 6  / 정오표 : [0, 1, 1, 1, 0, 1, 0, 0, 1, 1]
1번째 가중치 평균 값 : 0.582
2번째 맞춘 개수 : 6  / 정오표 : [0, 1, 1, 1, 1, 1, 0, 1, 0, 0]
2번째 가중치 평균 값 : 0.691
3번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 0, 1, 1, 1, 1, 1, 0]
3번째 가중치 평균 값 : 0.855
4번째 맞춘 개수 : 7  / 정오표 : [0, 1, 1, 1, 1, 1, 0, 1, 1, 0]
4번째 가중치 평균 값 : 0.727
5번째 맞춘 개수 : 7  / 정오표 : [0, 1, 1, 0, 1, 1, 0, 1, 1, 1]
5번째 가중치 평균 값 : 0.618
6번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
6번째 가중치 평균 값 : 0.909
7번째 맞춘 개수 : 6  / 정오표 : [0, 1, 1, 0, 1, 1, 0, 0, 1, 1]
7번째 가중치 평균 값 : 0.564
8번째 맞춘 개수 : 8  / 정오표 : [0, 1, 1, 1, 1, 1, 0, 1, 1, 1]
8번째 가중치 평균 값 : 0.745
9번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
9번째 가중치 평균 값 : 0.8
10번째 맞춘 개수 : 7  / 정오표 : [0, 1, 1, 1, 1, 0, 1, 1, 0, 1]
10번째 가중치 평균 값 : 0.691
Weighted Average Precision@K for question: 0.72


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
16208,16208,TRAIN_16208,./image/train/TRAIN_16208.jpg,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.<br>,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.,음식,음식점,한식,한식,"[-0.065323845, -0.07872713, 0.74007475, -0.019...",0.675734
1489,1489,TRAIN_01489,./image/train/TRAIN_01489.jpg,평창 대관령면에 위치한 카페로 커피가 정말 맛있고 계속 생각나는 맛이다.,평창 대관령면에 위치한 카페로 커피가 정말 맛있고 계속 생각나는 맛이다.,음식,음식점,바/까페,바/까페,"[-0.5000098, 0.41047162, 0.18789378, 0.0901001...",0.612291
13894,13894,TRAIN_13894,./image/train/TRAIN_13894.jpg,"부담 없이, 맘 편히 휴식하며, 맛있는 커피 한 잔의 여유를 즐길 수 있는 곳으로 ...","부담 없이, 맘 편히 휴식하며, 맛있는 커피 한 잔의 여유를 즐길 수 있는 곳으로 ...",음식,음식점,바/까페,바/까페,"[-0.5401062, 0.31217343, 0.44671902, 0.1881116...",0.599005
14010,14010,TRAIN_14010,./image/train/TRAIN_14010.jpg,커피와 건강 주스를 맛볼 수 있는 곳이다. 전라남도 목포시에 있는 카페다. 대표메뉴...,커피와 건강 주스를 맛볼 수 있는 곳이다. 전라남도 목포시에 있는 카페다. 대표메뉴...,음식,음식점,바/까페,바/까페,"[-0.3025178, 0.2914967, 0.23234744, 0.10090549...",0.564017
9489,9489,TRAIN_09489,./image/train/TRAIN_09489.jpg,평창 대관령면에 위치한 아늑한 분위기 좋은 대관령 마실 카페이다.,평창 대관령면에 위치한 아늑한 분위기 좋은 대관령 마실 카페이다.,음식,음식점,바/까페,바/까페,"[-0.32194695, 0.1385522, 0.8762667, 0.0441175,...",0.546321
6079,6079,TRAIN_06079,./image/train/TRAIN_06079.jpg,봉평면에 위치한 생 원두를 태우지 않고 직접 로스팅을 통해 추출하는 커피 전문점이다.,봉평면에 위치한 생 원두를 태우지 않고 직접 로스팅을 통해 추출하는 커피 전문점이다.,음식,음식점,바/까페,바/까페,"[-0.31039193, 0.52823466, -0.062238004, -0.072...",0.529273
3030,3030,TRAIN_03030,./image/train/TRAIN_03030.jpg,홈페이지 참조,홈페이지 참조,인문(문화/예술/역사),공연/행사,기타행사,축제/공연/행사,"[0.2027652, 0.14053588, 0.6965719, -0.04203199...",0.528859
7612,7612,TRAIN_07612,./image/train/TRAIN_07612.jpg,서울특별시 종로에 있는 카페이다. 탑골공원 근처에 있는 매장이다. 대표메뉴는 커피이다.,서울특별시 종로에 있는 카페이다. 탑골공원 근처에 있는 매장이다. 대표메뉴는 커피이다.,음식,음식점,바/까페,바/까페,"[-0.7033904, 0.25212613, 0.42847204, 0.0448072...",0.526003
10438,10438,TRAIN_10438,./image/train/TRAIN_10438.jpg,직접내린 커피를 맛볼 수 있는 곳이다. 대표메뉴는 아메리카노다. 서울특별시 영등포구...,직접내린 커피를 맛볼 수 있는 곳이다. 대표메뉴는 아메리카노다. 서울특별시 영등포구...,음식,음식점,바/까페,바/까페,"[-0.8515835, 0.22452603, 0.22341469, 0.2376924...",0.523050
2570,2570,TRAIN_02570,./image/train/TRAIN_02570.jpg,동해시 천곡동에 위치하였으며 도루묵 전문식당으로 유명하다.,동해시 천곡동에 위치하였으며 도루묵 전문식당으로 유명하다.,음식,음식점,한식,한식,"[-0.05982764, 0.17517398, 0.53054136, -0.15591...",0.517484


In [ ]:
true_label = '바/까페'
query_input = '커피와 디저트 파는 카페'
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

# doc2vec
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  1.0
전체데이터 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
전체데이터 가중치 평균 값 : 1.0

1번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1번째 가중치 평균 값 : 1.0
2번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2번째 가중치 평균 값 : 1.0
3번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
3번째 가중치 평균 값 : 1.0
4번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
4번째 가중치 평균 값 : 1.0
5번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
5번째 가중치 평균 값 : 1.0
6번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
6번째 가중치 평균 값 : 1.0
7번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
7번째 가중치 평균 값 : 1.0
8번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
8번째 가중치 평균 값 : 1.0
9번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
9번째 가중치 평균 값 : 1.0
10번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
10번째 가중치 평균 값 : 1.0
Weighted Average Precision@K for question: 1.0


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
1542,1542,TRAIN_01542,./image/train/TRAIN_01542.jpg,"카페 로시는 장해 율하 카페거리에 있는 핸드드립, 로스팅 커피 전문점이다. 다양한...","카페 로시는 장해 율하 카페거리에 있는 핸드드립, 로스팅 커피 전문점이다. 다양한...",음식,음식점,바/까페,바/까페,"[-0.76555395, 0.3096381, 0.25554532, 0.0118120...",0.816063
5337,5337,TRAIN_05337,./image/train/TRAIN_05337.jpg,"cafe G는 장해 율하 카페거리에 있는 브런치, 디저트 제공 카페이다. 간단한 식...","cafe G는 장해 율하 카페거리에 있는 브런치, 디저트 제공 카페이다. 간단한 식...",음식,음식점,바/까페,바/까페,"[-0.9861059, 0.43152496, 0.19417179, 0.0267367...",0.796300
9187,9187,TRAIN_09187,./image/train/TRAIN_09187.jpg,강원도 원주에 위치한 더 커피클럽에는 기다란 바(bar)가 자리하고 있다. 커피를 ...,강원도 원주에 위치한 더 커피클럽에는 기다란 바(bar)가 자리하고 있다. 커피를 ...,음식,음식점,바/까페,바/까페,"[-0.8469258, 0.36609238, 0.12158469, 0.1138252...",0.744938
13991,13991,TRAIN_13991,./image/train/TRAIN_13991.jpg,커피와 다양한 케이크를 맛볼 수 있다. 경기도 평택시에 있는 카페다. 대표메뉴는 커피다.,커피와 다양한 케이크를 맛볼 수 있다. 경기도 평택시에 있는 카페다. 대표메뉴는 커피다.,음식,음식점,바/까페,바/까페,"[-0.84509766, 0.24462967, -0.15677096, 0.15223...",0.741658
9378,9378,TRAIN_09378,./image/train/TRAIN_09378.jpg,커피도마시고 박물관도 구경하고 산책도 할 수 있는 다양한 즐길거리가 있는 문화공간 ...,커피도마시고 박물관도 구경하고 산책도 할 수 있는 다양한 즐길거리가 있는 문화공간 ...,음식,음식점,바/까페,바/까페,"[-0.58541775, 0.3525609, 0.54944384, -0.058975...",0.739041
11319,11319,TRAIN_11319,./image/train/TRAIN_11319.jpg,"커피나 차(茶), 음료 및 디저트류와 함께 간단한 식사가 가능한 달다(daldha)...","커피나 차(茶), 음료 및 디저트류와 함께 간단한 식사가 가능한 달다(daldha)...",음식,음식점,바/까페,바/까페,"[-0.5905496, 0.19067682, 0.10072621, 0.2268018...",0.733704
7957,7957,TRAIN_07957,./image/train/TRAIN_07957.jpg,소소한 재미가 있는 이스트덜위치는 인스타 감성카페로 꽃으로 디스플레이 해주는 han...,소소한 재미가 있는 이스트덜위치는 인스타 감성카페로 꽃으로 디스플레이 해주는 han...,음식,음식점,바/까페,바/까페,"[-0.9076008, 0.5671731, 0.43888235, 0.04382675...",0.732452
7612,7612,TRAIN_07612,./image/train/TRAIN_07612.jpg,서울특별시 종로에 있는 카페이다. 탑골공원 근처에 있는 매장이다. 대표메뉴는 커피이다.,서울특별시 종로에 있는 카페이다. 탑골공원 근처에 있는 매장이다. 대표메뉴는 커피이다.,음식,음식점,바/까페,바/까페,"[-0.7033904, 0.25212613, 0.42847204, 0.0448072...",0.723668
1489,1489,TRAIN_01489,./image/train/TRAIN_01489.jpg,평창 대관령면에 위치한 카페로 커피가 정말 맛있고 계속 생각나는 맛이다.,평창 대관령면에 위치한 카페로 커피가 정말 맛있고 계속 생각나는 맛이다.,음식,음식점,바/까페,바/까페,"[-0.5000098, 0.41047162, 0.18789378, 0.0901001...",0.719878
16041,16041,TRAIN_16041,./image/train/TRAIN_16041.jpg,"깊이 있는 커피와 차를 통해 한적한 정취를 즐기며 힐링하는 곳이며, 봉평 재래시장에...","깊이 있는 커피와 차를 통해 한적한 정취를 즐기며 힐링하는 곳이며, 봉평 재래시장에...",음식,음식점,바/까페,바/까페,"[-1.2059163, 0.3148862, 0.03372137, 0.11938538...",0.711977


NameError: name 'doc2vec_model' is not defined

In [ ]:
true_label = '쇼핑'
query_input = '전통시장이 있는 곳'
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

# doc2vec
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  1.0
전체데이터 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
전체데이터 가중치 평균 값 : 1.0

1번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1번째 가중치 평균 값 : 1.0
2번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2번째 가중치 평균 값 : 1.0
3번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
3번째 가중치 평균 값 : 1.0
4번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
4번째 가중치 평균 값 : 1.0
5번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
5번째 가중치 평균 값 : 1.0
6번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
6번째 가중치 평균 값 : 0.982
7번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
7번째 가중치 평균 값 : 1.0
8번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
8번째 가중치 평균 값 : 1.0
9번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
9번째 가중치 평균 값 : 1.0
10번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
10번째 가중치 평균 값 : 1.0
Weighted Average Precision@K for question: 1.0


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
5182,5182,TRAIN_05182,./image/train/TRAIN_05182.jpg,"동인천 전철역 뒤편 철길 주변에 조성된 시장으로, 동인천 지하상가 등이 인접해 있으...","동인천 전철역 뒤편 철길 주변에 조성된 시장으로, 동인천 지하상가 등이 인접해 있으...",쇼핑,쇼핑,상설시장,쇼핑,"[-0.0615403, -0.23017447, 0.38513583, 0.020734...",0.610341
15167,15167,TRAIN_15167,./image/train/TRAIN_15167.jpg,서천군 북부의 큰 시장으로 겨울철에는 해산물 특히 해태의 거래가 많았다. 오늘날에는...,서천군 북부의 큰 시장으로 겨울철에는 해산물 특히 해태의 거래가 많았다. 오늘날에는...,쇼핑,쇼핑,5일장,쇼핑,"[-0.7553146, -0.24314095, 0.8558692, 0.0534619...",0.608481
16133,16133,TRAIN_16133,./image/train/TRAIN_16133.jpg,1978년 설립된 의정부 제일시장은 점포 수 600개가 넘는 대형 전통시장이다. 시...,1978년 설립된 의정부 제일시장은 점포 수 600개가 넘는 대형 전통시장이다. 시...,쇼핑,쇼핑,상설시장,쇼핑,"[0.16421828, -0.29153618, 0.29853132, 0.113646...",0.585686
13126,13126,TRAIN_13126,./image/train/TRAIN_13126.jpg,사창시장은 충북 청주시 서원구에 자리 잡고 있다. 사창이란 명칭은 정부양곡을 보관하...,사창시장은 충북 청주시 서원구에 자리 잡고 있다. 사창이란 명칭은 정부양곡을 보관하...,쇼핑,쇼핑,상설시장,쇼핑,"[-0.18728876, -0.14252183, 0.2923867, 0.185589...",0.576871
16888,16888,TRAIN_16888,./image/train/TRAIN_16888.jpg,김제장은 일제 때부터 장이 형성되어 현재까지 운영되고 있는 전통시장이다. 현재는 전...,김제장은 일제 때부터 장이 형성되어 현재까지 운영되고 있는 전통시장이다. 현재는 전...,쇼핑,쇼핑,5일장,쇼핑,"[-0.103599355, -0.30672204, 0.72409874, -0.056...",0.564723
15746,15746,TRAIN_15746,./image/train/TRAIN_15746.jpg,기장시장은 기장읍 대라리를 중심으로 위치하고 있으며 시장 현대화 계획에 따라 198...,기장시장은 기장읍 대라리를 중심으로 위치하고 있으며 시장 현대화 계획에 따라 198...,쇼핑,쇼핑,상설시장,쇼핑,"[-0.34536573, -0.27368128, 0.9863604, -0.11101...",0.563988
8112,8112,TRAIN_08112,./image/train/TRAIN_08112.jpg,"세화민속오일시장은 세화 중심가 사거리에서 인접해 있어서 교통이 편리하고, 주민들과 ...","세화민속오일시장은 세화 중심가 사거리에서 인접해 있어서 교통이 편리하고, 주민들과 ...",쇼핑,쇼핑,5일장,쇼핑,"[-0.27312374, -0.39171502, 0.20133512, 0.07089...",0.558864
15859,15859,TRAIN_15859,./image/train/TRAIN_15859.jpg,1910년대에 개시된 전통깊은 시장으로 5일장과 상설시장의 형태로 열리고 있으며 정...,1910년대에 개시된 전통깊은 시장으로 5일장과 상설시장의 형태로 열리고 있으며 정...,쇼핑,쇼핑,상설시장,쇼핑,"[-0.5024072, -0.3813398, 0.93101716, -0.014366...",0.556388
1041,1041,TRAIN_01041,./image/train/TRAIN_01041.jpg,강북구 가오리역 근처에 위치한 골목형 소형시장이다.,강북구 가오리역 근처에 위치한 골목형 소형시장이다.,쇼핑,쇼핑,상설시장,쇼핑,"[-0.3288503, -0.50854695, 0.6673305, -0.026951...",0.554403
14555,14555,TRAIN_14555,./image/train/TRAIN_14555.jpg,구로동에 위치한 남구로시장은 1960년대 자연발생적으로 생긴 전통시장으로 50년 역...,구로동에 위치한 남구로시장은 1960년대 자연발생적으로 생긴 전통시장으로 50년 역...,쇼핑,쇼핑,상설시장,쇼핑,"[-0.27566987, -0.26403102, 0.4424971, 0.156929...",0.551819


NameError: name 'doc2vec_model' is not defined

In [ ]:
true_label = '한식'
query_input = '신선한 회를 파는 가게'
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

# doc2vec
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.6
전체데이터 맞춘 개수 : 6  / 정오표 : [1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
전체데이터 가중치 평균 값 : 0.691

1번째 맞춘 개수 : 6  / 정오표 : [1, 1, 1, 0, 0, 0, 1, 1, 0, 1]
1번째 가중치 평균 값 : 0.636
2번째 맞춘 개수 : 5  / 정오표 : [1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
2번째 가중치 평균 값 : 0.582
3번째 맞춘 개수 : 6  / 정오표 : [1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
3번째 가중치 평균 값 : 0.673
4번째 맞춘 개수 : 6  / 정오표 : [1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
4번째 가중치 평균 값 : 0.673
5번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 0, 0, 1, 1, 1, 1, 1]
5번째 가중치 평균 값 : 0.764
6번째 맞춘 개수 : 6  / 정오표 : [1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
6번째 가중치 평균 값 : 0.618
7번째 맞춘 개수 : 7  / 정오표 : [1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
7번째 가중치 평균 값 : 0.727
8번째 맞춘 개수 : 6  / 정오표 : [1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
8번째 가중치 평균 값 : 0.582
9번째 맞춘 개수 : 7  / 정오표 : [1, 1, 0, 0, 1, 0, 1, 1, 1, 1]
9번째 가중치 평균 값 : 0.636
10번째 맞춘 개수 : 6  / 정오표 : [1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
10번째 가중치 평균 값 : 0.6
Weighted Average Precision@K for question: 0.65


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
16400,16400,TRAIN_16400,./image/train/TRAIN_16400.jpg,동해시 묵호진동에 위치하였고 인근에 묵호항이 있으며 갓 잡아올린 싱싱한 재료를 사용...,동해시 묵호진동에 위치하였고 인근에 묵호항이 있으며 갓 잡아올린 싱싱한 재료를 사용...,음식,음식점,한식,한식,"[0.32762754, -0.10602443, 0.3227168, 0.0652792...",0.673444
450,450,TRAIN_00450,./image/train/TRAIN_00450.jpg,전통비법으로 음식을 만드는 매장이다. 전라남도 목포시에 있는 한식 전문점이다. 대표...,전통비법으로 음식을 만드는 매장이다. 전라남도 목포시에 있는 한식 전문점이다. 대표...,음식,음식점,한식,한식,"[-0.027810046, 0.20145583, 0.22068141, 0.09882...",0.670938
16208,16208,TRAIN_16208,./image/train/TRAIN_16208.jpg,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.<br>,낙지전골 전문으로 싱싱한 낙지를 사용해서 담백하고 얼큰하다.,음식,음식점,한식,한식,"[-0.065323845, -0.07872713, 0.74007475, -0.019...",0.664215
13456,13456,TRAIN_13456,./image/train/TRAIN_13456.jpg,"한우 숯불구이, 갈비탕 등을 판매하는 한식 전문점이다.","한우 숯불구이, 갈비탕 등을 판매하는 한식 전문점이다.",음식,음식점,한식,한식,"[-0.18075965, 0.3155509, 0.253468, 0.008687515...",0.645984
9032,9032,TRAIN_09032,./image/train/TRAIN_09032.jpg,당일생산 당일판매 천연 발효종으로 만든 건강하고 신선한 빵을 판매한다.,당일생산 당일판매 천연 발효종으로 만든 건강하고 신선한 빵을 판매한다.,음식,음식점,바/까페,바/까페,"[-0.5421707, 0.1095159, -0.4000374, -0.1493626...",0.639304
1555,1555,TRAIN_01555,./image/train/TRAIN_01555.jpg,통영 문화마당과 남망산 공원 입구 사이에 위치하고 있으며 매장 규모가 크고 다양한 ...,통영 문화마당과 남망산 공원 입구 사이에 위치하고 있으며 매장 규모가 크고 다양한 ...,쇼핑,쇼핑,특산물판매점,쇼핑,"[-0.25679383, 0.27988425, 0.29570693, 0.138443...",0.638070
11490,11490,TRAIN_11490,./image/train/TRAIN_11490.jpg,다양한 짬뽕요리를 먹을 수 있는 매장이다. 전라남도 목포시에 있는 중식전문점이다. ...,다양한 짬뽕요리를 먹을 수 있는 매장이다. 전라남도 목포시에 있는 중식전문점이다. ...,음식,음식점,중식,식사류,"[-0.13520987, 0.49242166, -0.017022997, 0.1040...",0.634510
5811,5811,TRAIN_05811,./image/train/TRAIN_05811.jpg,※ 영업시간 00:00 ~ 24:00\n\n20년동안 해장국만 전문으로 연구하고 요...,※ 영업시간 00:00 ~ 24:0020년동안 해장국만 전문으로 연구하고 요리하는 ...,음식,음식점,한식,한식,"[0.18521991, 0.09227147, 0.51261806, -0.044926...",0.633270
9205,9205,TRAIN_09205,./image/train/TRAIN_09205.jpg,백석마을에 위치한 순성면을 흐르는 남원천 가장자리에는 매실나무 수천그루가 줄을 서 ...,백석마을에 위치한 순성면을 흐르는 남원천 가장자리에는 매실나무 수천그루가 줄을 서 ...,인문(문화/예술/역사),체험관광지,농.산.어촌 체험,휴양/체험/산업관광지,"[-0.6263143, 0.32783863, 0.27349007, -0.047562...",0.632977
3649,3649,TRAIN_03649,./image/train/TRAIN_03649.jpg,고객과 함께 즐거움을 공유하는 것을 기업가치로 추구하는 (주)크래비언의 첫 브랜드 ...,고객과 함께 즐거움을 공유하는 것을 기업가치로 추구하는 (주)크래비언의 첫 브랜드 ...,음식,음식점,한식,한식,"[-0.78715795, -0.07325025, 0.29162866, 0.19239...",0.626595


NameError: name 'doc2vec_model' is not defined

In [ ]:
true_label = '식사류'
query_input = '짜장면이 맛있는 집'
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

# doc2vec
res = doc2vec_weighted_avg_precision(df, query_input, true_label,k=10)
print(res)
display(doc2vec_return_answer(query_input,df))

전체데이터 acc :  0.0
전체데이터 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.0

1번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.0
2번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2번째 가중치 평균 값 : 0.0
3번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3번째 가중치 평균 값 : 0.0
4번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4번째 가중치 평균 값 : 0.0
5번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5번째 가중치 평균 값 : 0.0
6번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6번째 가중치 평균 값 : 0.0
7번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.0
8번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8번째 가중치 평균 값 : 0.0
9번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.0
10번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10번째 가중치 평균 값 : 0.0
Weighted Average Precision@K for question: 0.0


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
2730,2730,TRAIN_02730,./image/train/TRAIN_02730.jpg,"빈티지도 좋지만 환경까지 생각한다면 빈티지 프레임부터 테라리움 장식장, 버려진 와인...","빈티지도 좋지만 환경까지 생각한다면 빈티지 프레임부터 테라리움 장식장, 버려진 와인...",쇼핑,쇼핑,전문상가,쇼핑,"[-0.32407713, 0.3897224, 0.54225796, 0.2126932...",0.738884
13836,13836,TRAIN_13836,./image/train/TRAIN_13836.jpg,통영펜션은 통영 인터페인지 인근 언덕위에 위치하여 전망이 아주 좋다. 공기도 맑고 ...,통영펜션은 통영 인터페인지 인근 언덕위에 위치하여 전망이 아주 좋다. 공기도 맑고 ...,숙박,숙박시설,펜션,숙박,"[-0.3176627, 0.0729887, 0.6593288, 0.38243347,...",0.737930
16340,16340,TRAIN_16340,./image/train/TRAIN_16340.jpg,금농헌은 가야금 소리가 무르 익어가는 아름다운 집으로 다양한 한옥 문화를 체험할 수...,금농헌은 가야금 소리가 무르 익어가는 아름다운 집으로 다양한 한옥 문화를 체험할 수...,숙박,숙박시설,홈스테이,숙박,"[0.19876467, 0.49542573, 0.6239278, -0.1726949...",0.737257
7028,7028,TRAIN_07028,./image/train/TRAIN_07028.jpg,평창군에 위치한 숙박업소로 주변에 편의시설과 아름다운 자연환경을 갖추고 있다.<br...,평창군에 위치한 숙박업소로 주변에 편의시설과 아름다운 자연환경을 갖추고 있다.깨끗한...,숙박,숙박시설,펜션,숙박,"[-0.3679516, 0.31196228, 0.9444789, 0.150815, ...",0.737066
2470,2470,TRAIN_02470,./image/train/TRAIN_02470.jpg,전통과 현대가 어우러진 고즈넉한 한옥 숙소이다.,전통과 현대가 어우러진 고즈넉한 한옥 숙소이다.,숙박,숙박시설,홈스테이,숙박,"[-0.08771517, -0.0821382, 0.98729205, 0.285370...",0.735180
2144,2144,TRAIN_02144,./image/train/TRAIN_02144.jpg,지리산 끌림 펜션은 주변 경관이 뛰어난 곳에 자리하여 아름다운 경치와 맑은 공기를 ...,지리산 끌림 펜션은 주변 경관이 뛰어난 곳에 자리하여 아름다운 경치와 맑은 공기를 ...,숙박,숙박시설,펜션,숙박,"[-0.38165513, 0.19241941, 0.9861191, 0.3205408...",0.725390
13906,13906,TRAIN_13906,./image/train/TRAIN_13906.jpg,1993년쯤 문을 열었다. 깔끔한 실내와 각종 조형물 인테리어로 편안한 분위기에서 ...,1993년쯤 문을 열었다. 깔끔한 실내와 각종 조형물 인테리어로 편안한 분위기에서 ...,음식,음식점,한식,한식,"[-0.08455869, 0.2949022, 0.76914465, 0.0309742...",0.725098
655,655,TRAIN_00655,./image/train/TRAIN_00655.jpg,꾸미지 않은 듯한 편안한 분위기에서 개운하고 얼큰한 메기매운탕을 즐길 수 있는 곳이...,꾸미지 않은 듯한 편안한 분위기에서 개운하고 얼큰한 메기매운탕을 즐길 수 있는 곳이다.,음식,음식점,한식,한식,"[-0.43323582, -0.22680303, 0.77799237, 0.28336...",0.716593
11493,11493,TRAIN_11493,./image/train/TRAIN_11493.jpg,"자연이 머무는 곳, 마음이 쉬어가는 곳. 강원도 강릉에는 다양한 형태의 한옥이 모여...","자연이 머무는 곳, 마음이 쉬어가는 곳. 강원도 강릉에는 다양한 형태의 한옥이 모여...",숙박,숙박시설,한옥스테이,숙박,"[0.10371095, 0.75072503, 0.91785467, 0.0318951...",0.715493
7635,7635,TRAIN_07635,./image/train/TRAIN_07635.jpg,한적하고 공기 좋은 곳에 위치한 숙박업소이다.,한적하고 공기 좋은 곳에 위치한 숙박업소이다.,숙박,숙박시설,홈스테이,숙박,"[0.1063797, -0.1300938, 0.7710882, 0.22747545,...",0.713275


NameError: name 'doc2vec_model' is not defined

In [ ]:
true_label = '레포츠'
query_input = '캠핑장'
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

전체데이터 acc :  0.9
전체데이터 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
전체데이터 가중치 평균 값 : 0.982

1번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1번째 가중치 평균 값 : 1.0
2번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
2번째 가중치 평균 값 : 0.873
3번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
3번째 가중치 평균 값 : 0.964
4번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
4번째 가중치 평균 값 : 0.945
5번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
5번째 가중치 평균 값 : 0.964
6번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
6번째 가중치 평균 값 : 1.0
7번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
7번째 가중치 평균 값 : 0.945
8번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
8번째 가중치 평균 값 : 0.927
9번째 맞춘 개수 : 9  / 정오표 : [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
9번째 가중치 평균 값 : 0.927
10번째 맞춘 개수 : 10  / 정오표 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
10번째 가중치 평균 값 : 1.0
Weighted Average Precision@K for question: 0.95


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
1694,1694,TRAIN_01694,./image/train/TRAIN_01694.jpg,진동리 국민여가캠핑장 by zapza는 강원도 인제군 기린면에 자리 잡고 있다. 캠...,진동리 국민여가캠핑장 by zapza는 강원도 인제군 기린면에 자리 잡고 있다. 캠...,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.23409139, 0.63824695, 0.20664892, -0.148643...",0.668651
4418,4418,TRAIN_04418,./image/train/TRAIN_04418.jpg,삼척 엘림캠핑장은 강원도 삼척시 근덕면 내평길 233-13에 위치하고 있다. 수량이...,삼척 엘림캠핑장은 강원도 삼척시 근덕면 내평길 233-13에 위치하고 있다. 수량이...,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[-0.029136699, 0.17176257, 0.3500624, 0.014711...",0.663549
16697,16697,TRAIN_16697,./image/train/TRAIN_16697.jpg,평창 봉평면에 위치한 곳으로 흥정계곡 옆에 있으며 조용하고 깨끗한 시설의 캠핑장이다.,평창 봉평면에 위치한 곳으로 흥정계곡 옆에 있으며 조용하고 깨끗한 시설의 캠핑장이다.,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.05258528, 0.23645924, 0.65639514, -0.116490...",0.659616
12640,12640,TRAIN_12640,./image/train/TRAIN_12640.jpg,강변사리마을캠핑장은 전북 임실군 덕치면 강변사리마을 내에 위치한 캠핑장이다. 강변...,강변사리마을캠핑장은 전북 임실군 덕치면 강변사리마을 내에 위치한 캠핑장이다. 강변...,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.35858744, 0.2913716, 0.64240485, -0.1850753...",0.653337
5390,5390,TRAIN_05390,./image/train/TRAIN_05390.jpg,푸른자연과 함께하는 한적한 가족캠핑장,푸른자연과 함께하는 한적한 가족캠핑장,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.044100694, -0.24878138, 0.3444356, 0.056655...",0.625670
79,79,TRAIN_00079,./image/train/TRAIN_00079.jpg,여름에도 시원한 산타파크캠핑장은 강원도 태백시 황지동에 위치하고 있다. 노지캠핑장으...,여름에도 시원한 산타파크캠핑장은 강원도 태백시 황지동에 위치하고 있다. 노지캠핑장으...,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.14411402, 0.3067278, 0.5231762, 0.12734373,...",0.625431
11189,11189,TRAIN_11189,./image/train/TRAIN_11189.jpg,오가리 캠핑장은 한탄강 줄기가 흐르는 곳에 있는 펜션과 야영장이 함께 있는 캠핑장이...,오가리 캠핑장은 한탄강 줄기가 흐르는 곳에 있는 펜션과 야영장이 함께 있는 캠핑장이...,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.29128945, 0.25198874, 0.36015773, 0.2071804...",0.620544
13518,13518,TRAIN_13518,./image/train/TRAIN_13518.jpg,양평숲 비룡 캠프장은 경기도 양평군 청운면에 자리 잡고 있다. 캠핑장에는 데크와 파...,양평숲 비룡 캠프장은 경기도 양평군 청운면에 자리 잡고 있다. 캠핑장에는 데크와 파...,레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.14169762, 0.18238233, 0.56544465, 0.2327213...",0.620110
3591,3591,TRAIN_03591,./image/train/TRAIN_03591.jpg,"숲속가든 캠핑장은 사천 와룡산 자락에 자리를 잡고 있으며, 사천시 시내에서 고성군으...","숲속가든 캠핑장은 사천 와룡산 자락에 자리를 잡고 있으며, 사천시 시내에서 고성군으...",레포츠,육상 레포츠,"야영장,오토캠핑장",레포츠,"[0.4144013, 0.06624949, 0.34259224, 0.16815606...",0.618225
3387,3387,TRAIN_03387,./image/train/TRAIN_03387.jpg,산모랭이펜션은 진부면에 위치하고 있다.,산모랭이펜션은 진부면에 위치하고 있다.,숙박,숙박시설,펜션,숙박,"[-0.068702996, -0.08789411, 0.81154406, 0.0506...",0.611018


In [ ]:
true_label = '자연'
query_input = '바다가 예쁜 해수욕장'
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

전체데이터 acc :  0.5
전체데이터 맞춘 개수 : 5  / 정오표 : [0, 0, 1, 0, 1, 0, 1, 1, 1, 0]
전체데이터 가중치 평균 값 : 0.418

1번째 맞춘 개수 : 5  / 정오표 : [0, 0, 1, 0, 1, 1, 1, 0, 0, 1]
1번째 가중치 평균 값 : 0.436
2번째 맞춘 개수 : 6  / 정오표 : [0, 0, 1, 0, 1, 1, 1, 0, 1, 1]
2번째 가중치 평균 값 : 0.473
3번째 맞춘 개수 : 6  / 정오표 : [1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
3번째 가중치 평균 값 : 0.618
4번째 맞춘 개수 : 5  / 정오표 : [0, 0, 1, 1, 0, 1, 1, 1, 0, 0]
4번째 가중치 평균 값 : 0.491
5번째 맞춘 개수 : 5  / 정오표 : [0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
5번째 가중치 평균 값 : 0.509
6번째 맞춘 개수 : 5  / 정오표 : [0, 0, 1, 1, 0, 0, 1, 1, 0, 1]
6번째 가중치 평균 값 : 0.418
7번째 맞춘 개수 : 5  / 정오표 : [0, 0, 1, 0, 1, 0, 1, 1, 1, 0]
7번째 가중치 평균 값 : 0.418
8번째 맞춘 개수 : 5  / 정오표 : [0, 0, 1, 0, 1, 1, 1, 0, 0, 1]
8번째 가중치 평균 값 : 0.436
9번째 맞춘 개수 : 7  / 정오표 : [0, 1, 1, 1, 1, 0, 0, 1, 1, 1]
9번째 가중치 평균 값 : 0.655
10번째 맞춘 개수 : 6  / 정오표 : [0, 0, 1, 0, 1, 1, 1, 0, 1, 1]
10번째 가중치 평균 값 : 0.473
Weighted Average Precision@K for question: 0.49


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
15784,15784,TRAIN_15784,./image/train/TRAIN_15784.jpg,"변산반도의 서쪽 해안자락에는 변산 해수욕장, 격포 해수욕장 등 아름다운 해수욕장이 ...","변산반도의 서쪽 해안자락에는 변산 해수욕장, 격포 해수욕장 등 아름다운 해수욕장이 ...",인문(문화/예술/역사),체험관광지,농.산.어촌 체험,휴양/체험/산업관광지,"[0.056864098, -0.590697, 0.13306056, 0.5067108...",0.828421
7576,7576,TRAIN_07576,./image/train/TRAIN_07576.jpg,"맑은물소리 펜션은 내린천 맑은 물에서 낚시, 래프팅을 즐기고 멱도 감을 수 있는 곳...","맑은물소리 펜션은 내린천 맑은 물에서 낚시, 래프팅을 즐기고 멱도 감을 수 있는 곳...",숙박,숙박시설,펜션,숙박,"[-0.09291666, -0.29271224, 0.28030905, 0.69830...",0.815632
6717,6717,TRAIN_06717,./image/train/TRAIN_06717.jpg,지리적 형상이 좋고 나무가 우거져 도인들이 도를 닦던 마을이라 하여 도여라 불렀으며...,지리적 형상이 좋고 나무가 우거져 도인들이 도를 닦던 마을이라 하여 도여라 불렀으며...,자연,자연관광지,해수욕장,자연,"[-0.1459121, -0.33769464, 0.45459512, 0.614611...",0.805354
16011,16011,TRAIN_16011,./image/train/TRAIN_16011.jpg,예래생태체험축제는 '물과 함께 신명나는 해변축제'를 주제로 천혜의 경관을 자랑하는 ...,예래생태체험축제는 '물과 함께 신명나는 해변축제'를 주제로 천혜의 경관을 자랑하는 ...,인문(문화/예술/역사),축제,일반축제,축제/공연/행사,"[0.078466825, -0.34119514, 0.83666384, 0.06548...",0.796726
13285,13285,TRAIN_13285,./image/train/TRAIN_13285.jpg,바위로만 뒤덮인 욕지도에는 모래해변이 거의 없다. 유동·덕동·흰작살 등의 해수욕장에...,바위로만 뒤덮인 욕지도에는 모래해변이 거의 없다. 유동·덕동·흰작살 등의 해수욕장에...,자연,자연관광지,해수욕장,자연,"[-0.08031504, -0.12536986, 0.1833499, 0.573247...",0.795134
9837,9837,TRAIN_09837,./image/train/TRAIN_09837.jpg,솔향기가가득한집 펜션은 걸어서 바닷가까지 불과 5분 거리에 위치한 펜션으로 여름철 ...,솔향기가가득한집 펜션은 걸어서 바닷가까지 불과 5분 거리에 위치한 펜션으로 여름철 ...,숙박,숙박시설,펜션,숙박,"[-0.1650124, -0.44628695, 0.4152699, 0.2613223...",0.794240
11029,11029,TRAIN_11029,./image/train/TRAIN_11029.jpg,"'명사(明沙)'라는 지명에서도 알 수 있듯이, 모래의 질이 좋고 물이 맑은 해수욕장...","'명사(明沙)'라는 지명에서도 알 수 있듯이, 모래의 질이 좋고 물이 맑은 해수욕장...",자연,자연관광지,해수욕장,자연,"[0.011822305, -0.305618, -0.12695095, 0.124504...",0.793515
6293,6293,TRAIN_06293,./image/train/TRAIN_06293.jpg,"* 넓은 갯벌, 금빛 해변이 갈매기 날개처럼 펼쳐지는 독산해수욕장<br /><br ...","* 넓은 갯벌, 금빛 해변이 갈매기 날개처럼 펼쳐지는 독산해수욕장충남 보령시 웅천읍...",자연,자연관광지,해수욕장,자연,"[0.16220868, -0.26118392, 0.3353874, 0.3668820...",0.789597
14987,14987,TRAIN_14987,./image/train/TRAIN_14987.jpg,상록수림과 해안바위가 조화를 이룬 아름다운 섬으로 가 서이말 등대에서 바라보면 거북...,상록수림과 해안바위가 조화를 이룬 아름다운 섬으로 가 서이말 등대에서 바라보면 거북...,자연,자연관광지,섬,자연,"[-0.3104743, -0.42726323, 0.4767259, 0.0273846...",0.789289
1425,1425,TRAIN_01425,./image/train/TRAIN_01425.jpg,산방산과 송악산의 숨쉬는 마을 바닷가하우스는 해안도로에 근접해 있어 주변 경지가 일...,산방산과 송악산의 숨쉬는 마을 바닷가하우스는 해안도로에 근접해 있어 주변 경지가 일...,숙박,숙박시설,펜션,숙박,"[-0.14054641, -0.27131706, 0.961071, 0.3025093...",0.778957


In [ ]:
true_label = '바/까페'
query_input = df['overview'][10787]
# bert
res = weighted_avg_precision(df, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df))

전체데이터 acc :  0.7
전체데이터 맞춘 개수 : 7  / 정오표 : [1, 1, 1, 1, 0, 0, 1, 1, 1, 0]
전체데이터 가중치 평균 값 : 0.782

1번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 0, 0, 1, 1, 1, 1, 1]
1번째 가중치 평균 값 : 0.764
2번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 0, 1, 1, 0, 1, 1, 1]
2번째 가중치 평균 값 : 0.8
3번째 맞춘 개수 : 8  / 정오표 : [1, 1, 0, 1, 1, 0, 1, 1, 1, 1]
3번째 가중치 평균 값 : 0.764
4번째 맞춘 개수 : 7  / 정오표 : [1, 1, 0, 0, 1, 1, 1, 0, 1, 1]
4번째 가중치 평균 값 : 0.673
5번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
5번째 가중치 평균 값 : 0.818
6번째 맞춘 개수 : 7  / 정오표 : [1, 1, 1, 0, 0, 1, 1, 1, 0, 1]
6번째 가중치 평균 값 : 0.727
7번째 맞춘 개수 : 8  / 정오표 : [1, 1, 0, 0, 1, 1, 1, 1, 1, 1]
7번째 가중치 평균 값 : 0.727
8번째 맞춘 개수 : 7  / 정오표 : [1, 1, 0, 0, 1, 1, 0, 1, 1, 1]
8번째 가중치 평균 값 : 0.655
9번째 맞춘 개수 : 8  / 정오표 : [1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
9번째 가중치 평균 값 : 0.8
10번째 맞춘 개수 : 9  / 정오표 : [1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
10번째 가중치 평균 값 : 0.855
Weighted Average Precision@K for question: 0.76


,Unnamed: 0,id,img_path,overview,text,cat1,cat2,cat3,cat,SBERT_emb,score
10787,10787,TRAIN_10787,./image/train/TRAIN_10787.jpg,오창카페식물원은 카페 이름처럼 작은 식물원을 연상시키는 곳이다. 카페 공간 어느 곳...,오창카페식물원은 카페 이름처럼 작은 식물원을 연상시키는 곳이다. 카페 공간 어느 곳...,음식,음식점,바/까페,바/까페,"[-0.59379596, 0.6053004, -0.2295956, 0.1163554...",1.000000
4106,4106,TRAIN_04106,./image/train/TRAIN_04106.jpg,제주시 송당리에 위치한 송당나무는 꽃과 식물이 가득한 가드닝 센터다. 강수량이 많은...,제주시 송당리에 위치한 송당나무는 꽃과 식물이 가득한 가드닝 센터다. 강수량이 많은...,음식,음식점,바/까페,바/까페,"[-0.77573043, -0.023008225, 0.0629878, 0.30611...",0.830824
349,349,TRAIN_00349,./image/train/TRAIN_00349.jpg,기흥저수지 근처에 있는 고풍스러운 대형 온실 카페이다. 고급스러운 건물에서 커피를 ...,기흥저수지 근처에 있는 고풍스러운 대형 온실 카페이다. 고급스러운 건물에서 커피를 ...,음식,음식점,바/까페,바/까페,"[-1.0448823, 0.003853857, 0.38331574, 0.066208...",0.812369
6587,6587,TRAIN_06587,./image/train/TRAIN_06587.jpg,"크래커스는 원두를 직접 로스팅하는 곳으로 유명한 커피 맛집이다. 한경점, 대정점 두...","크래커스는 원두를 직접 로스팅하는 곳으로 유명한 커피 맛집이다. 한경점, 대정점 두...",음식,음식점,바/까페,바/까페,"[-0.5904115, 0.7040851, -0.057404064, 0.230877...",0.802052
16782,16782,TRAIN_16782,./image/train/TRAIN_16782.jpg,이효석생가 바로 옆에 위치한 메밀꽃필무렵은 입구에서부터 아기자기한 소품들이 눈에 띈...,이효석생가 바로 옆에 위치한 메밀꽃필무렵은 입구에서부터 아기자기한 소품들이 눈에 띈...,음식,음식점,한식,한식,"[-0.65060055, 0.25475004, 0.33773035, 0.009140...",0.790251
9944,9944,TRAIN_09944,./image/train/TRAIN_09944.jpg,경기도 광주에 있는 설렁탕 전문점이다. 이곳엔 음식으로 치료할 수 없는 병은 약으로...,경기도 광주에 있는 설렁탕 전문점이다. 이곳엔 음식으로 치료할 수 없는 병은 약으로...,음식,음식점,한식,한식,"[-0.6526894, 0.25476533, -0.2573447, 0.2380624...",0.778705
15814,15814,TRAIN_15814,./image/train/TRAIN_15814.jpg,로스터리 카페 유동커피는 서귀포 올레 시장 근처에 있다. 서귀포 본점을 비롯하여 전...,로스터리 카페 유동커피는 서귀포 올레 시장 근처에 있다. 서귀포 본점을 비롯하여 전...,음식,음식점,바/까페,바/까페,"[-0.6252732, 0.5096313, -0.06698324, 0.0880301...",0.775468
8170,8170,TRAIN_08170,./image/train/TRAIN_08170.jpg,"모아이(MOANI)는 바람에 실려 오는 향기란 뜻으로, 두 번째 자연이라는 이름 아...","모아이(MOANI)는 바람에 실려 오는 향기란 뜻으로, 두 번째 자연이라는 이름 아...",음식,음식점,바/까페,바/까페,"[-0.8564195, 0.51102686, -0.22816876, 0.048389...",0.773821
13368,13368,TRAIN_13368,./image/train/TRAIN_13368.jpg,설문커피는 고양시 설문동에 있는 야외정원이 카페다. 모두 한 가족이 운영하는 설문추...,설문커피는 고양시 설문동에 있는 야외정원이 카페다. 모두 한 가족이 운영하는 설문추...,음식,음식점,바/까페,바/까페,"[-0.74976015, 0.71326685, -0.08568964, -0.1112...",0.769186
5329,5329,TRAIN_05329,./image/train/TRAIN_05329.jpg,소백산꽃차이야기는 영주의 귀농 여성들로 구성 된 체험 농장형 꽃차 카페이다. 아름답...,소백산꽃차이야기는 영주의 귀농 여성들로 구성 된 체험 농장형 꽃차 카페이다. 아름답...,인문(문화/예술/역사),체험관광지,이색거리,휴양/체험/산업관광지,"[-0.69143915, -0.00044846698, 0.26317364, -0.1...",0.767099


### bert2

In [ ]:
true_label = '쇼핑'
query_input = '전통시장이 있는 곳'
# bert
res = weighted_avg_precision(df2, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df2))

전체데이터 acc :  0.0
전체데이터 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.0

1번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.0
2번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
2번째 가중치 평균 값 : 0.0
3번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
3번째 가중치 평균 값 : 0.0
4번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
4번째 가중치 평균 값 : 0.0
5번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
5번째 가중치 평균 값 : 0.0
6번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
6번째 가중치 평균 값 : 0.0
7번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.0
8번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
8번째 가중치 평균 값 : 0.0
9번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.0
10번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
10번째 가중치 평균 값 : 0.0
Weighted Average Precision@K for question: 0.0


,Unnamed: 0,id,img_path,overview,cat1,cat2,cat3,cat,embedding,score
13086,13086,TRAIN_13086,./image/train/TRAIN_13086.jpg,제10회 서울도시농업박람회는 온·오프라인에서 동시 개최된다.,인문(문화/예술/역사),공연/행사,박람회,축제/공연/행사,"[-1.4625671, -0.10295486, 0.44589758, -1.06564...",0.950688
10247,10247,TRAIN_10247,./image/train/TRAIN_10247.jpg,상주시 외답동에 위치한 손짜장 전문점이다.,음식,음식점,중식,식사류,"[-1.3275744, -0.4355798, 0.38675594, -1.195003...",0.949274
9139,9139,TRAIN_09139,./image/train/TRAIN_09139.jpg,청풍호에 위치한 약선요리정식 전문점이다.,음식,음식점,한식,한식,"[-1.5948099, -0.15956095, 0.6151657, -1.269247...",0.945191
14507,14507,TRAIN_14507,./image/train/TRAIN_14507.jpg,다양한 쟝르 청년 예술인들의 공연 축제,인문(문화/예술/역사),축제,일반축제,축제/공연/행사,"[-1.658295, -0.33131847, 0.76225835, -1.084424...",0.943784
14695,14695,TRAIN_14695,./image/train/TRAIN_14695.jpg,쫄면이 맛있는 분식 전문점이다.,음식,음식점,한식,한식,"[-1.3624529, -0.40596166, 0.38022712, -1.16990...",0.942947


In [ ]:
true_label = '식사류'
query_input = '짜장면이 맛있는 집'
# bert2
res = weighted_avg_precision(df2, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df2))

전체데이터 acc :  0.0
전체데이터 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
전체데이터 가중치 평균 값 : 0.0

1번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
1번째 가중치 평균 값 : 0.0
2번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
2번째 가중치 평균 값 : 0.0
3번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 0, 1]
3번째 가중치 평균 값 : 0.067
4번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
4번째 가중치 평균 값 : 0.0
5번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 1, 0]
5번째 가중치 평균 값 : 0.133
6번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 0, 1]
6번째 가중치 평균 값 : 0.067
7번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
7번째 가중치 평균 값 : 0.0
8번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
8번째 가중치 평균 값 : 0.0
9번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
9번째 가중치 평균 값 : 0.0
10번째 맞춘 개수 : 0  / 정오표 : [0, 0, 0, 0, 0]
10번째 가중치 평균 값 : 0.0
Weighted Average Precision@K for question: 0.03


,Unnamed: 0,id,img_path,overview,cat1,cat2,cat3,cat,embedding,score
12570,12570,TRAIN_12570,./image/train/TRAIN_12570.jpg,양구에 위치한 민박이다.,숙박,숙박시설,펜션,숙박,"[-1.3571669, -0.39888966, 0.4963091, -1.318601...",0.849938
577,577,TRAIN_00577,./image/train/TRAIN_00577.jpg,자꾸 먹고싶은 옛날 치킨의 맛! 진부면에 위치한 일미통닭이다.,음식,음식점,한식,한식,"[-1.337289, -0.2717675, 0.2830503, -1.1795167,...",0.837807
5405,5405,TRAIN_05405,./image/train/TRAIN_05405.jpg,양양읍 윌리에 위치한 강촌식당은 뚜거리탕과 은어회의 맛깔스러움을 느낄 수 있는 곳이...,음식,음식점,한식,한식,"[-1.405062, -0.18619691, 0.47255456, -0.926514...",0.809603
11463,11463,TRAIN_11463,./image/train/TRAIN_11463.jpg,휘닉스에메랄드는 평창군 봉평면에 위치하고 있다.,숙박,숙박시설,펜션,숙박,"[-1.4813259, -0.2062997, 0.54207957, -1.125495...",0.794941
14695,14695,TRAIN_14695,./image/train/TRAIN_14695.jpg,쫄면이 맛있는 분식 전문점이다.,음식,음식점,한식,한식,"[-1.3624529, -0.40596166, 0.38022712, -1.16990...",0.790320


In [ ]:
true_label = '한식'
query_input = '신선한 회를 파는 가게'
# bert2
res = weighted_avg_precision(df2, query_input, true_label)
print(res)
display(bert_return_answer(query_input,df2))


전체데이터 acc :  0.4
전체데이터 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 0, 1]
전체데이터 가중치 평균 값 : 0.333

1번째 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 1, 0]
1번째 가중치 평균 값 : 0.4
2번째 맞춘 개수 : 3  / 정오표 : [0, 1, 1, 0, 1]
2번째 가중치 평균 값 : 0.533
3번째 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 0, 1]
3번째 가중치 평균 값 : 0.333
4번째 맞춘 개수 : 1  / 정오표 : [0, 0, 0, 1, 0]
4번째 가중치 평균 값 : 0.133
5번째 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 0, 1]
5번째 가중치 평균 값 : 0.333
6번째 맞춘 개수 : 2  / 정오표 : [0, 0, 1, 0, 1]
6번째 가중치 평균 값 : 0.267
7번째 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 1, 0]
7번째 가중치 평균 값 : 0.4
8번째 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 0, 1]
8번째 가중치 평균 값 : 0.333
9번째 맞춘 개수 : 2  / 정오표 : [0, 1, 0, 1, 0]
9번째 가중치 평균 값 : 0.4
10번째 맞춘 개수 : 2  / 정오표 : [0, 0, 1, 0, 1]
10번째 가중치 평균 값 : 0.267
Weighted Average Precision@K for question: 0.34


,Unnamed: 0,id,img_path,overview,cat1,cat2,cat3,cat,embedding,score
3387,3387,TRAIN_03387,./image/train/TRAIN_03387.jpg,산모랭이펜션은 진부면에 위치하고 있다.,숙박,숙박시설,펜션,숙박,"[-1.2943411, -0.46354085, 0.3485022, -1.226955...",0.965535
13388,13388,TRAIN_13388,./image/train/TRAIN_13388.jpg,안동 시내에 위치한 한정식 전문점이다.,음식,음식점,한식,한식,"[-1.2983049, -0.44770506, 0.38186792, -1.37754...",0.953502
5684,5684,TRAIN_05684,./image/train/TRAIN_05684.jpg,펜션 앞에 펼처진 멋진 산세와 그 아래 깨끗하고 맑은 금당계곡이 있다.,숙박,숙박시설,펜션,숙박,"[-1.4057738, -0.3277901, 0.3874078, -1.2354627...",0.950890
8483,8483,TRAIN_08483,./image/train/TRAIN_08483.jpg,메이페어는 평창군 봉평면에 위치하고 있다.,숙박,숙박시설,펜션,숙박,"[-1.4030645, -0.42463806, 0.57779104, -1.15118...",0.945639
5033,5033,TRAIN_05033,./image/train/TRAIN_05033.jpg,신선한 활어회 제철 해산물을 다양한 밑반찬과 함께 즐길 수 있다.,음식,음식점,한식,한식,"[-1.370061, -0.50740933, 0.4060387, -1.1909181...",0.943188
